In [1]:
import numpy as np
from math import gamma
import matplotlib.pyplot as plt
from scipy.optimize import minimize

import pandas as pd

import os
os.chdir("C:\\Users\\user\\Downloads\\Thesis_paper_1\\calibration")

np.set_printoptions(suppress=True) # supress scientific notation

import matplotlib.ticker as mtick
import locale
locale.setlocale(locale.LC_NUMERIC, "de_DE")
plt.rcParams['axes.formatter.use_locale'] = True

pd.set_option('display.max_columns', 50) 

In [2]:

import tensorflow as tf
import tensorflow_probability as tfp

In [3]:
## parameters



def get_data(year):
    w='data_' + str(year)
    alfa = tf.convert_to_tensor(np.array(pd.read_excel(f'{w}.xlsx').iloc[:,5], dtype='float64').reshape(1,3) )
    phi = tf.convert_to_tensor(np.array(pd.read_excel(f'{w}.xlsx').iloc[:,4], dtype='float64').reshape(1,3) )
    return alfa, phi



def pars(year):
    global alfa, rho, beta, theta, eta, psi, kappa, lamb, gamma1, G, phi, s
    rho = tf.constant(0.19, tf.float64)
    beta = tf.constant(0.231, tf.float64)
    theta = tf.constant(2.0, tf.float64)
    eta = tf.constant(0.103, tf.float64)
    psi = eta**eta*(1.0-eta)**(1.0-eta)
    kappa = tf.divide(1.0, (1.0 - eta))
    lamb = 1 - tf.divide(kappa, theta)
    gamma1 = tf.multiply(eta, kappa)    
    G = tf.constant(gamma(1 - tf.divide(kappa, tf.multiply(theta, (1-rho)) ) ), tf.float64  )      
    alfa, phi = get_data(year)
    s = tf.pow( (1 + tf.divide( (1-eta), ( tf.multiply(beta, phi) ) ) ), -1 )


pars(15)

In [4]:
# taus
st=3
def taus(st):        
    h_bar = tf.random.uniform(maxval=10, minval=-10, shape=[1, st], dtype='float64')
    x1 = np.array([h_bar])
    x1 = tf.convert_to_tensor(x1, tf.float64)
    return x1

In [5]:
x1[0]

In [6]:
x1=taus(3)

In [7]:
x1[0]

In [8]:
def N_if(x1):
    S = tf.pow(s, phi*(theta*lamb+ kappa) )
    S2 =tf.pow((1 - s), ( (theta*lamb)/(beta*kappa) ) )
    H = tf.pow((1 - x1[0]), (theta*lamb + kappa))    
    C = tf.multiply(tf.multiply(tf.pow(psi, theta*lamb), eta**gamma1 ), G )
    N_i = tf.multiply(S, tf.multiply(S2, tf.multiply(H, C)) )
    return N_i

In [9]:
N_if(x1)

In [10]:

#  linear system 

def matrix():
    global A, B
    A = np.repeat(alfa, st).reshape(st, st).T       # Matriz de parâmetros    
    B = np.repeat(alfa, st).reshape(st, st).T        # Matriz de alfas 
    for i in range(st):
        for j in range(st):
            if i==j:
                A[i, j] = 1 - (A[i, j] - 1)*(theta*lamb + gamma1)
                B[i, j] = B[i, j] - 1
            if i != j:
                A[i, j] = -A[i, j]*(theta*lamb + gamma1) 
    A = tf.convert_to_tensor(A, tf.float64)
    B = tf.convert_to_tensor(B, tf.float64)
    

def lin_sys(x1):
    #global N_i
    #print(f'x1 = {x1}')
    N_i = N_if(x1)
    N_i2 = tf.math.log( N_i )                                                    # Logaritmo de N_i
    C = tf.math.log(alfa)                                                        # Logaritmo dos alfas
    D = tf.reshape(tf.tensordot(B, tf.reshape(N_i2, (3,1)) , 1), (1, 3) ) + C    # produto matricial                     
    res = tf.reshape(tf.exp(tf.linalg.solve(A, tf.reshape(D, (3,1)) ) ), (1,3) )  # Resolvendo o sistema  A w = D
    #res =tf.reshape(res, (1,3))
    return res

In [11]:
matrix()
lin_sys(x1)

In [12]:
def w_til_f(x1):
    #w = lin_sys(x1)
    #print(f'w = {w}')
    S = tf.pow(s, phi)
    S2 = tf.pow( (1 - s), 1/(beta*kappa) )
    H = 1 - x1[0]
    w_til = tf.multiply(psi, tf.multiply(H, tf.multiply(S, tf.multiply(w , S2))))
    return w_til

   

In [13]:
# wages - model 

def W_i_f(x1):
    w_til = w_til_f(x1)
    sum_wtil = tf.pow( tf.reduce_sum( tf.pow(w_til, theta) ) , (kappa/theta) ) 
    S = tf.pow((1-s), (-1/beta))
    p_i = tf.pow(w_til, theta)/ tf.reduce_sum( tf.pow(w_til, theta) )   
    W_i = tf.multiply(kappa, tf.multiply(G, tf.multiply(S, tf.multiply(sum_wtil, p_i)) ) )    
    return W_i, p_i

In [14]:
# human capital

def hc_f(x1):
    W = tf.pow(w , (theta*lamb + gamma1) )
    W_til = tf.pow(tf.reduce_sum(tf.pow(w_til_f(x1), theta) ), lamb)
    N_i = N_if(x1)
    Hc = tf.multiply(N_i, tf.divide(W, W_til))
    return Hc

In [15]:
# PNAD - DATA


def get_p_i(year):
    global p_s2
    w='data_' + str(year)
    p_i = tf.convert_to_tensor(np.array(pd.read_excel(f'{w}.xlsx').iloc[:,2]).reshape(1,3), tf.float64)
    p_s2 = p_i[1:st]
    return p_i


def get_W_i(year):
    p_i = get_p_i(year)
    global W_id
    w='data_' + str(year)
    W_id = tf.convert_to_tensor(np.multiply(np.array(pd.read_excel(f'{w}.xlsx').iloc[:,1]), p_i).reshape(1,3), tf.float64)
    return W_id

In [16]:
#@tf.function
def tflow(x1):    
    global w    
    #w = tf.reshape(lin_sys(x1), (1, 3) )
    w=lin_sys(x1)
    x1=tf.reshape(x1, (1, 1, 3))
    #x1 = np.reshape(x1, (2,3))
    tf.convert_to_tensor(x1, tf.float64)
    W_im = W_i_f(x1)[0]    
    D = tf.reduce_sum(tf.pow( tf.divide(tf.math.squared_difference(W_id, W_im), W_id ) , 2)) 
    D = tf.math.log(D)
    return D

In [17]:
#@tf.function
def tflow(x1):    
    global w    
    #w = tf.reshape(lin_sys(x1), (1, 3) )
    w=lin_sys(x1)
    x1=tf.reshape(x1, (1, 3))
    #x1 = np.reshape(x1, (2,3))
    tf.convert_to_tensor(x1, tf.float64)
    W_im = W_i_f(x1)[0]    
    D = tf.reduce_sum(tf.pow( tf.divide(tf.math.squared_difference(W_id, W_im), W_id ) , 2)) 
    D = tf.math.log(D)
    return D

In [18]:
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [19]:
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [20]:
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [21]:
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [22]:
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [23]:
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [24]:
@tf.function
def tflow(x1):    
    global w    
    #w = tf.reshape(lin_sys(x1), (1, 3) )
    w=lin_sys(x1)
    x1=tf.reshape(x1, (1, 3))
    #x1 = np.reshape(x1, (2,3))
    tf.convert_to_tensor(x1, tf.float64)
    W_im = W_i_f(x1)[0]    
    D = tf.reduce_sum(tf.pow( tf.divide(tf.math.squared_difference(W_id, W_im), W_id ) , 2)) 
    D = tf.math.log(D)
    return D

In [25]:
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [26]:
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [27]:
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [28]:
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [29]:
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [30]:
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [31]:
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [32]:
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [33]:
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [34]:
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [35]:
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [36]:
# bounds


Bd =  ((0, 1e-5), )*st + ((-20, 20), )*st 

Bd = np.array(Bd, dtype=object)

In [37]:
# callback


cc = 0
def callback(x):
    global cc
    cc += 1
    fobj = tflow(x)
    print(f'\033[1;033mObjetivo: {np.around(fobj, 10)}, iter: {cc}') 

In [38]:
st = 3
ano= 15
x1 = taus(st)
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

#optim_results = tfp.optimizer.lbfgs_minimize(
#      tflow,
#      x1,
#      num_correction_pairs=10,
#      tolerance=1e-8)


dtype= x1.dtype
start =tf.constant(x1, dtype=dtype)


sol = tfp.optimizer.nelder_mead_minimize(
  tflow,
  initial_vertex=start,
  func_tolerance=1e-20)

In [39]:
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

In [40]:
st = 3
x1 = taus(st)
ano= 95
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

sol= minimize(tflow, x1,  method='L-BFGS-B', bounds=Bd,
              callback=callback, parallel_iterations =10,
              tol=1e-250, options={'maxiter':1e3}) 

In [41]:
st = 3
x1 = taus(st)
ano= 95
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

sol= minimize(tflow, x1,  method='L-BFGS-B', bounds=Bd,
              callback=callback,
              tol=1e-250, options={'maxiter':1e3}) 

In [42]:
st = 3
x1 = taus(st)
ano= 95
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

sol= minimize(tflow, x1,  method='L-BFGS-B', bounds=Bd,
              callback=callback,
              tol=1e-250, options={'maxiter':1e3}) 

In [43]:
# bounds


Bd =   ((-20, 20), )*st 

Bd = np.array(Bd, dtype=object)

In [44]:
st = 3
x1 = taus(st)
ano= 95
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

sol= minimize(tflow, x1,  method='L-BFGS-B', bounds=Bd,
              callback=callback,
              tol=1e-250, options={'maxiter':1e3}) 

In [45]:
sol.x    
 
# ----> 95   2.71376964, -0.12759258, 2.13168792

In [46]:
st = 3
ano= 15
x1 = taus(st)
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

#optim_results = tfp.optimizer.lbfgs_minimize(
#      tflow,
#      x1,
#      num_correction_pairs=10,
#      tolerance=1e-8)


dtype= x1.dtype
start =tf.constant(x1, dtype=dtype)


sol = tfp.optimizer.nelder_mead_minimize(
  tflow,
  initial_vertex=start,
  parallel_iterations =10
  func_tolerance=1e-20)

In [47]:
st = 3
ano= 15
x1 = taus(st)
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

#optim_results = tfp.optimizer.lbfgs_minimize(
#      tflow,
#      x1,
#      num_correction_pairs=10,
#      tolerance=1e-8)


dtype= x1.dtype
start =tf.constant(x1, dtype=dtype)


sol = tfp.optimizer.nelder_mead_minimize(
  tflow,
  initial_vertex=start,
  parallel_iterations =10,
  func_tolerance=1e-20)

In [48]:
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

In [49]:
st = 3
ano= 95
x1 = taus(st)
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

#optim_results = tfp.optimizer.lbfgs_minimize(
#      tflow,
#      x1,
#      num_correction_pairs=10,
#      tolerance=1e-8)


dtype= x1.dtype
start =tf.constant(x1, dtype=dtype)


sol = tfp.optimizer.nelder_mead_minimize(
  tflow,
  initial_vertex=start,
  parallel_iterations =10,
  func_tolerance=1e-20)

In [50]:
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

In [51]:
st = 3
ano= 95
x1 = taus(st)
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

#optim_results = tfp.optimizer.lbfgs_minimize(
#      tflow,
#      x1,
#      num_correction_pairs=10,
#      tolerance=1e-8)


dtype= x1.dtype
start =tf.constant(x1, dtype=dtype)


sol = tfp.optimizer.nelder_mead_minimize(
  tflow,
  initial_vertex=start,
  parallel_iterations =10,
  func_tolerance=1e-20)

In [52]:
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

In [53]:
st = 3
ano= 15
x1 = taus(st)
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

#optim_results = tfp.optimizer.lbfgs_minimize(
#      tflow,
#      x1,
#      num_correction_pairs=10,
#      tolerance=1e-8)


dtype= x1.dtype
start =tf.constant(x1, dtype=dtype)


sol = tfp.optimizer.nelder_mead_minimize(
  tflow,
  initial_vertex=start,
  parallel_iterations =10,
  func_tolerance=1e-20)

In [54]:
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

In [55]:
st = 3
ano= 15
x1 = taus(st)
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

#optim_results = tfp.optimizer.lbfgs_minimize(
#      tflow,
#      x1,
#      num_correction_pairs=10,
#      tolerance=1e-8)


dtype= x1.dtype
start =tf.constant(x1, dtype=dtype)


sol = tfp.optimizer.nelder_mead_minimize(
  tflow,
  initial_vertex=start,
  parallel_iterations =10,
  func_tolerance=1e-200)

In [56]:
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

In [57]:
st = 3
ano= 15
x1 = taus(st)
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

#optim_results = tfp.optimizer.lbfgs_minimize(
#      tflow,
#      x1,
#      num_correction_pairs=10,
#      tolerance=1e-8)


dtype= x1.dtype
start =tf.constant(x1, dtype=dtype)


sol = tfp.optimizer.nelder_mead_minimize(
  tflow,
  initial_vertex=start,
  parallel_iterations =10,
  func_tolerance=1e-200)

In [58]:
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

In [59]:
st = 3
ano= 15
x1 = taus(st)
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

#optim_results = tfp.optimizer.lbfgs_minimize(
#      tflow,
#      x1,
#      num_correction_pairs=10,
#      tolerance=1e-8)


dtype= x1.dtype
start =tf.constant(x1, dtype=dtype)


sol = tfp.optimizer.nelder_mead_minimize(
  tflow,
  initial_vertex=start,
  parallel_iterations =10,
    shrinkage=0.3,
  func_tolerance=1e-200)

In [60]:
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

In [61]:
st = 3
ano= 15
x1 = taus(st)
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

#optim_results = tfp.optimizer.lbfgs_minimize(
#      tflow,
#      x1,
#      num_correction_pairs=10,
#      tolerance=1e-8)


dtype= x1.dtype
start =tf.constant(x1, dtype=dtype)


sol = tfp.optimizer.nelder_mead_minimize(
  tflow,
  initial_vertex=start,
  parallel_iterations =10,
    shrinkage=0.3,
  func_tolerance=1e-200)

In [62]:
tfp.optimizer.lbfgs_minimize(
  tflow,
  initial_position=start,
  parallel_iterations =10,
    shrinkage=0.3,
  func_tolerance=1e-200)

In [63]:
tfp.optimizer.lbfgs_minimize(
  tflow,
  initial_position=start,
  parallel_iterations =10,
  func_tolerance=1e-200)

In [64]:
tfp.optimizer.lbfgs_minimize(
  tflow,
  initial_position=start,
  parallel_iterations =10,
  tolerance=1e-200)

In [65]:
start = np.random.randn(st)

tfp.optimizer.lbfgs_minimize(
  tflow,
  initial_position=start,
  parallel_iterations =10,
  tolerance=1e-200)

In [66]:
tf.test.is_gpu_available(cuda_only = False) 

In [67]:
def make_val_and_grad_fn(value_fn):
  @functools.wraps(value_fn)
  def val_and_grad(x):
    return tfp.math.value_and_gradient(value_fn, x)
  return val_and_grad

In [68]:
#@tf.function
def tflow(x1):    
    global w    
    #w = tf.reshape(lin_sys(x1), (1, 3) )
    w=lin_sys(x1)
    x1=tf.reshape(x1, (1, 3))
    #x1 = np.reshape(x1, (2,3))
    tf.convert_to_tensor(x1, tf.float64)
    W_im = W_i_f(x1)[0]    
    D = tf.reduce_sum(tf.pow( tf.divide(tf.math.squared_difference(W_id, W_im), W_id ) , 2)) 
    D = tf.math.log(D)
    return D

In [69]:
@tf.function
def tflow2(x1):    
    global w    
    #w = tf.reshape(lin_sys(x1), (1, 3) )
    w=lin_sys(x1)
    x1=tf.reshape(x1, (1, 3))
    #x1 = np.reshape(x1, (2,3))
    tf.convert_to_tensor(x1, tf.float64)
    W_im = W_i_f(x1)[0]    
    D = tf.reduce_sum(tf.pow( tf.divide(tf.math.squared_difference(W_id, W_im), W_id ) , 2)) 
    D = tf.math.log(D)
    return D

In [70]:
@tf.function
def tflow2(x1):    
    global w    
    #w = tf.reshape(lin_sys(x1), (1, 3) )
    w=lin_sys(x1)
    x1=tf.reshape(x1, (1, 3))
    #x1 = np.reshape(x1, (2,3))
    tf.convert_to_tensor(x1, tf.float64)
    W_im = W_i_f(x1)[0]    
    D = tf.reduce_sum(tf.pow( tf.divide(tf.math.squared_difference(W_id, W_im), W_id ) , 2)) 
    D = tf.math.log(D)
    return D

In [71]:
def make_val_and_grad_fn(value_fn):
  @functools.wraps(value_fn)
  def val_and_grad(x):
    return tfp.math.value_and_gradient(value_fn, x)
  return val_and_grad



@contextlib.contextmanager
def timed_execution():
  t0 = time.time()
  yield
  dt = time.time() - t0
  print('Evaluation took: %f seconds' % dt)


def np_value(tensor):
  """Get numpy value out of possibly nested tuple of tensors."""
  if isinstance(tensor, tuple):
    return type(tensor)(*(np_value(t) for t in tensor))
  else:
    return tensor.numpy()

def run(optimizer):
  """Run an optimizer and measure it's evaluation time."""
  optimizer()  # Warmup.
  with timed_execution():
    result = optimizer()
  return np_value(result)

In [72]:

import tensorflow as tf
import tensorflow_probability as tfp

import contextlib
import functools

In [73]:
def make_val_and_grad_fn(value_fn):
  @functools.wraps(value_fn)
  def val_and_grad(x):
    return tfp.math.value_and_gradient(value_fn, x)
  return val_and_grad



@contextlib.contextmanager
def timed_execution():
  t0 = time.time()
  yield
  dt = time.time() - t0
  print('Evaluation took: %f seconds' % dt)


def np_value(tensor):
  """Get numpy value out of possibly nested tuple of tensors."""
  if isinstance(tensor, tuple):
    return type(tensor)(*(np_value(t) for t in tensor))
  else:
    return tensor.numpy()

def run(optimizer):
  """Run an optimizer and measure it's evaluation time."""
  optimizer()  # Warmup.
  with timed_execution():
    result = optimizer()
  return np_value(result)

In [74]:
start = np.random.randn(st)

In [75]:
def tflow2(x1):    
    global w    
    #w = tf.reshape(lin_sys(x1), (1, 3) )
    w=lin_sys(x1)
    x1=tf.reshape(x1, (1, 3))
    #x1 = np.reshape(x1, (2,3))
    tf.convert_to_tensor(x1, tf.float64)
    W_im = W_i_f(x1)[0]    
    D = tf.reduce_sum(tf.pow( tf.divide(tf.math.squared_difference(W_id, W_im), W_id ) , 2)) 
    D = tf.math.log(D)
    return D

In [76]:
@tf.function
def quadratic_with_lbfgs():
  return tfp.optimizer.lbfgs_minimize(
  tflow,
  initial_position=start,
  parallel_iterations =10,
  tolerance=1e-200)

sol = run(quadratic_with_lbfgs)

In [77]:
def tflow2(x1):    
    global w    
    #w = tf.reshape(lin_sys(x1), (1, 3) )
    w=lin_sys(x1)
    x1=tf.reshape(x1, (1, 3))
    #x1 = np.reshape(x1, (2,3))
    tf.convert_to_tensor(x1, tf.float64)
    W_im = W_i_f(x1)[0]    
    D = tf.reduce_sum(tf.pow( tf.divide(tf.math.squared_difference(W_id, W_im), W_id ) , 2)) 
    D = tf.math.log(D)
    return D

In [78]:
@tf.function
def quadratic_with_lbfgs():
  return tfp.optimizer.lbfgs_minimize(
  tflow2,
  initial_position=start,
  parallel_iterations =10,
  tolerance=1e-200)

sol = run(quadratic_with_lbfgs)

In [79]:
#@tf.function
def tflow(x1):    
    global w    
    #w = tf.reshape(lin_sys(x1), (1, 3) )
    w=lin_sys(x1)
    x1=tf.reshape(x1, (1, 3))
    #x1 = np.reshape(x1, (2,3))
    tf.convert_to_tensor(x1, tf.float64)
    W_im = W_i_f(x1)[0]    
    D = tf.reduce_sum(tf.pow( tf.divide(tf.math.squared_difference(W_id, W_im), W_id ) , 2)) 
    D = tf.math.log(D)
    return D

In [80]:
@tf.function
def quadratic_with_lbfgs():
  return tfp.optimizer.lbfgs_minimize(
  tflow2,
  initial_position=start,
  parallel_iterations =10,
  tolerance=1e-200)

sol = run(quadratic_with_lbfgs)

In [81]:
@tf.function
def quadratic_with_lbfgs():
  return tfp.optimizer.lbfgs_minimize(
  tflow2,
  initial_position=taus(3),
  parallel_iterations =10,
  tolerance=1e-200)

sol = run(quadratic_with_lbfgs)

In [82]:
@tf.function
def quadratic_with_lbfgs():
  return tfp.optimizer.lbfgs_minimize(
  tflow2,
  initial_position=start,
  parallel_iterations =10,
  tolerance=1e-200)

sol = run(quadratic_with_lbfgs)

In [83]:
@tf.function
def quadratic_with_lbfgs():
  return tfp.optimizer.lbfgs_minimize(
  tflow2,
  initial_position=tf.constant(np.array([0.1,1.0,2.0]), dtype=dtype),
  parallel_iterations =10,
  tolerance=1e-200)

sol = run(quadratic_with_lbfgs)

In [84]:
#@tf.function
def quadratic_with_lbfgs():
  return tfp.optimizer.lbfgs_minimize(
  tflow2,
  initial_position=tf.constant(np.array([0.1,1.0,2.0]), dtype=dtype),
  parallel_iterations =10,
  tolerance=1e-200)

sol = run(quadratic_with_lbfgs)

In [85]:
@make_val_and_grad_fn
def tflow2(x1):    
    global w    
    #w = tf.reshape(lin_sys(x1), (1, 3) )
    w=lin_sys(x1)
    x1=tf.reshape(x1, (1, 3))
    #x1 = np.reshape(x1, (2,3))
    tf.convert_to_tensor(x1, tf.float64)
    W_im = W_i_f(x1)[0]    
    D = tf.reduce_sum(tf.pow( tf.divide(tf.math.squared_difference(W_id, W_im), W_id ) , 2)) 
    D = tf.math.log(D)
    return D

In [86]:
@tf.function
def quadratic_with_lbfgs():
  return tfp.optimizer.lbfgs_minimize(
  tflow2,
  initial_position=tf.constant(np.array([0.1,1.0,2.0]), dtype=dtype),
  parallel_iterations =10,
  tolerance=1e-200)

sol = run(quadratic_with_lbfgs)

In [87]:
import numpy as np
from math import gamma
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import time
import pandas as pd

import os
os.chdir("C:\\Users\\user\\Downloads\\Thesis_paper_1\\calibration")

np.set_printoptions(suppress=True) # supress scientific notation

import matplotlib.ticker as mtick
import locale
locale.setlocale(locale.LC_NUMERIC, "de_DE")


plt.rcParams['axes.formatter.use_locale'] = True



pd.set_option('display.max_columns', 50) 

In [88]:
@tf.function
def quadratic_with_lbfgs():
  return tfp.optimizer.lbfgs_minimize(
  tflow2,
  initial_position=tf.constant(np.array([0.1,1.0,2.0]), dtype=dtype),
  parallel_iterations =10,
  tolerance=1e-200)

sol = run(quadratic_with_lbfgs)

In [89]:
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

In [90]:
print('LBFGS')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

In [91]:
@tf.function
def quadratic_with_lbfgs():
  return tfp.optimizer.lbfgs_minimize(
  tflow2,
  initial_position=tf.constant(np.array([0.1,1.0,2.0]), dtype=dtype),
  parallel_iterations =10,
  tolerance=1e-200)

sol = run(quadratic_with_lbfgs)

In [92]:
print('LBFGS')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

In [93]:
st = 3
x1 = taus(st)
ano= 95
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

sol= minimize(tflow, x1,  method='L-BFGS-B', bounds=Bd,
              callback=callback,
              tol=1e-250, options={'maxiter':1e3}) 

In [94]:
st = 3
x1 = taus(st)
ano= 15
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

sol= minimize(tflow, x1,  method='L-BFGS-B', bounds=Bd,
              callback=callback,
              tol=1e-250, options={'maxiter':1e3}) 

In [95]:
sol.x    
 
# ----> 95   2.71376964, -0.12759258, 2.13168792

In [96]:
st = 3
x1 = taus(st)
ano= 15
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

sol= minimize(tflow, x1,  method='L-BFGS-B', bounds=Bd,
              callback=callback,
              tol=1e-250, options={'maxiter':1e3}) 

In [97]:
sol.x    
 
# ----> 95   2.71376964, -0.12759258, 2.13168792

In [98]:
st = 3
x1 = taus(st)
ano= 15
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

sol= minimize(tflow, x1,  method='L-BFGS-B', bounds=Bd,
              callback=callback,
              tol=1e-250, options={'maxiter':1e3}) 

In [99]:
sol.x    
 
# ----> 95   2.71376964, -0.12759258, 2.13168792

In [100]:
sol.x    
 
# ----> 95 

In [101]:
@tf.function
def quadratic_with_lbfgs():
  return tfp.optimizer.lbfgs_minimize(
  tflow2,
  initial_position=tf.constant(np.array([0.1,1.0,2.0]), dtype=dtype),
  parallel_iterations =10,
  tolerance=1e-250,)

sol = run(quadratic_with_lbfgs)

In [102]:
@tf.function
def quadratic_with_lbfgs():
  return tfp.optimizer.lbfgs_minimize(
  tflow2,
  initial_position=tf.constant(np.array([0.1,1.0,2.0]), dtype=dtype),
  parallel_iterations =10,
  tolerance=1e-250,
    max_iterations=1e3, 
    max_line_search_iterations=1e4)

sol = run(quadratic_with_lbfgs)

In [103]:
@tf.function
def quadratic_with_lbfgs():
  return tfp.optimizer.lbfgs_minimize(
  tflow2,
  initial_position=tf.constant(np.array([0.1,1.0,2.0]), dtype=dtype),
  parallel_iterations =10,
  tolerance=1e-250,
    max_iterations=1e3
    )

sol = run(quadratic_with_lbfgs)

In [104]:
@tf.function
def quadratic_with_lbfgs():
  return tfp.optimizer.lbfgs_minimize(
  tflow2,
  initial_position=tf.constant(np.array([0.1,1.0,2.0]), dtype=dtype),
  parallel_iterations =10,
  tolerance=1e-250    )

sol = run(quadratic_with_lbfgs)

In [105]:
print('LBFGS')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

In [106]:
@tf.function
def quadratic_with_lbfgs():
  return tfp.optimizer.lbfgs_minimize(
  tflow2,
  initial_position=tf.constant(np.array([0.1,1.0,2.0]), dtype=dtype),
  parallel_iterations =10,
  tolerance=1e-250,
    max_iterations=1000)

sol = run(quadratic_with_lbfgs)

In [107]:
print('LBFGS')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

In [108]:
@tf.function
def quadratic_with_lbfgs():
  return tfp.optimizer.lbfgs_minimize(
  tflow2,
  initial_position=tf.constant(np.array([0.1,1.0,2.0]), dtype=dtype),
  parallel_iterations =10,
  tolerance=1e-250,
    max_iterations=10000)

sol = run(quadratic_with_lbfgs)

In [109]:
print('LBFGS')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

In [110]:
@tf.function
def quadratic_with_lbfgs():
  return tfp.optimizer.lbfgs_minimize(
  tflow2,
  initial_position=tf.constant(np.array([0.1,1.0,2.0]), dtype=dtype),
  parallel_iterations =10,
  tolerance=1e-250,
    max_iterations=10000,
  max_line_search_iterations=5000)

sol = run(quadratic_with_lbfgs)

In [111]:
print('LBFGS')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

In [112]:
@make_val_and_grad_fn
def tflow2(x1):    
    global w    
    x1=tf.reshape(x1, (1, 3))
    w=lin_sys(x1)
    tf.convert_to_tensor(x1, tf.float64)
    W_im = W_i_f(x1)[0]    
    D = tf.reduce_sum(tf.pow( tf.divide(tf.math.squared_difference(W_id, W_im), W_id ) , 2)) 
    D = tf.math.log(D)
    return D

In [113]:
@tf.function
def quadratic_with_lbfgs():
  return tfp.optimizer.lbfgs_minimize(
  tflow2,
  initial_position=tf.constant(np.array([0.1,1.0,2.0]), dtype=dtype),
  parallel_iterations =10,
  tolerance=1e-250,
    max_iterations=10000,
  max_line_search_iterations=5000)

sol = run(quadratic_with_lbfgs)

In [114]:
print('LBFGS')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

In [115]:
@tf.function
def quadratic_with_lbfgs():
  return tfp.optimizer.lbfgs_minimize(
  tflow2,
  initial_position=tf.constant(np.array([0.1,1.0,2.0]), dtype=dtype),
  parallel_iterations =10,
  tolerance=0,
    max_iterations=10000,
  max_line_search_iterations=5000)

sol = run(quadratic_with_lbfgs)

In [116]:
print('LBFGS')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

In [117]:
@make_val_and_grad_fn
def tflow2(x1):    
    global w    
    x1=tf.reshape(x1, (1, 3))
    w=lin_sys(x1)
    tf.convert_to_tensor(x1, tf.float64)
    W_im = W_i_f(x1)[0]    
    D = tf.reduce_sum(tf.pow( tf.divide(tf.math.squared_difference(W_id, W_im), W_id ) , 2)) 
    #D = tf.math.log(D)
    return D

In [118]:
@tf.function
def quadratic_with_lbfgs():
  return tfp.optimizer.lbfgs_minimize(
  tflow2,
  initial_position=tf.constant(np.array([0.1,1.0,2.0]), dtype=dtype),
  parallel_iterations =10,
  tolerance=0,
    max_iterations=10000,
  max_line_search_iterations=5000)

sol = run(quadratic_with_lbfgs)

In [119]:
print('LBFGS')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

In [120]:
@make_val_and_grad_fn
def tflow2(x1):    
    global w   
    w=lin_sys(x1)
    x1=tf.reshape(x1, (1, 3))
    tf.convert_to_tensor(x1, tf.float64)
    W_im = W_i_f(x1)[0]    
    D = tf.reduce_sum(tf.pow( tf.divide(tf.math.squared_difference(W_id, W_im), W_id ) , 2)) 
    #D = tf.math.log(D)
    return D

In [121]:
@tf.function
def quadratic_with_lbfgs():
  return tfp.optimizer.lbfgs_minimize(
  tflow2,
  initial_position=tf.constant(np.array([0.1,1.0,2.0]), dtype=dtype),
  parallel_iterations =10,
  tolerance=0,
    max_iterations=10000,
  max_line_search_iterations=5000)

sol = run(quadratic_with_lbfgs)

In [122]:
print('LBFGS')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

In [123]:
@tf.function
def quadratic_with_lbfgs():
  return tfp.optimizer.lbfgs_minimize(
  tflow2,
  initial_position=tf.constant(np.array([-10.0,5.0,2.0]), dtype=dtype),
  parallel_iterations =10,
  tolerance=0,
    max_iterations=10000,
  max_line_search_iterations=5000)

sol = run(quadratic_with_lbfgs)

In [124]:
print('LBFGS')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

In [125]:
st = 3
x1 = taus(st)
ano= 15
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

sol= minimize(tflow, x1,  method='L-BFGS-B', bounds=Bd,
              callback=callback,
              tol=1e-250, options={'maxiter':1e3}) 

In [126]:
sol.x    
 
# ----> 95 

In [127]:
st = 3
x1 = taus(st)
ano= 15
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

sol= minimize(tflow, x1,  method='L-BFGS-B', bounds=Bd,
              callback=callback,
              tol=1e-250, options={'maxiter':1e3}) 

In [128]:
sol.x    
 
# ----> 95 

In [129]:
#@tf.function
def tflow(x1):    
    global w    
    #w = tf.reshape(lin_sys(x1), (1, 3) )
    w=lin_sys(x1)
    x1=tf.reshape(x1, (1, 3))
    #x1 = np.reshape(x1, (2,3))
    tf.convert_to_tensor(x1, tf.float64)
    W_im = W_i_f(x1)[0]    
    D = tf.reduce_sum(tf.pow( tf.divide(tf.math.squared_difference(W_id, W_im), W_id ) , 2)) 
    #D = tf.math.log(D)
    return D

In [130]:
st = 3
x1 = taus(st)
ano= 15
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

sol= minimize(tflow, x1,  method='L-BFGS-B', bounds=Bd,
              callback=callback,
              tol=1e-250, options={'maxiter':1e3}) 

In [131]:
sol.x    
 
# ----> 95 

In [132]:
@tf.function
def quadratic_with_lbfgs():
  return tfp.optimizer.lbfgs_minimize(
  tflow2,
  initial_position=tf.constant(np.array([-10.0,5.0,-2.0]), dtype=dtype),
  parallel_iterations =10,
  tolerance=0,
    max_iterations=10000,
  max_line_search_iterations=5000)

sol = run(quadratic_with_lbfgs)

In [133]:
print('LBFGS')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

In [134]:
st = 3
ano= 15
x1 = taus(st)
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

#optim_results = tfp.optimizer.lbfgs_minimize(
#      tflow,
#      x1,
#      num_correction_pairs=10,
#      tolerance=1e-8)


dtype= x1.dtype
start = tf.constant(x1, dtype=dtype)


sol = tfp.optimizer.nelder_mead_minimize(
  tflow,
  initial_vertex=start,
  parallel_iterations =10,
  shrinkage=0.3,
  func_tolerance=1e-200)

In [135]:
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

In [136]:
st = 3
ano= 15
x1 = taus(st)
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

#optim_results = tfp.optimizer.lbfgs_minimize(
#      tflow,
#      x1,
#      num_correction_pairs=10,
#      tolerance=1e-8)


dtype= x1.dtype
start = tf.constant(x1, dtype=dtype)


def nelder_mead():
  return tfp.optimizer.nelder_mead_minimize(
  tflow,
  initial_vertex=start,
  parallel_iterations =10,
  shrinkage=0.3,
  func_tolerance=1e-200)


sol = run(nelder_mead)

In [137]:
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

In [138]:
st = 3
ano= 15
x1 = taus(st)
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

#optim_results = tfp.optimizer.lbfgs_minimize(
#      tflow,
#      x1,
#      num_correction_pairs=10,
#      tolerance=1e-8)


dtype= x1.dtype
start = tf.constant(x1, dtype=dtype)


def nelder_mead():
  return tfp.optimizer.nelder_mead_minimize(
  tflow,
  initial_vertex=start,
  parallel_iterations =10,
  shrinkage=0.3,
  func_tolerance=0)


sol = run(nelder_mead)

In [139]:
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

In [140]:
st = 3
ano= 15
x1 = taus(st)
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

#optim_results = tfp.optimizer.lbfgs_minimize(
#      tflow,
#      x1,
#      num_correction_pairs=10,
#      tolerance=1e-8)


dtype= x1.dtype
start = tf.constant(x1, dtype=dtype)


def nelder_mead():
  return tfp.optimizer.nelder_mead_minimize(
  tflow,
  initial_vertex=start,
  parallel_iterations =10,
  shrinkage=0.7,
  func_tolerance=0)


sol = run(nelder_mead)

In [141]:
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

In [142]:
sol = run(nelder_mead)
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

In [143]:
st = 3
ano= 15
x1 = taus(st)
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

#optim_results = tfp.optimizer.lbfgs_minimize(
#      tflow,
#      x1,
#      num_correction_pairs=10,
#      tolerance=1e-8)


dtype= x1.dtype
start = tf.constant(x1, dtype=dtype)


def nelder_mead():
  return tfp.optimizer.nelder_mead_minimize(
  tflow,
  initial_vertex=start,
  parallel_iterations =10,
  shrinkage=0.9,
  func_tolerance=0)

In [144]:
sol = run(nelder_mead)
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

In [145]:
sol = run(nelder_mead)
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

In [146]:
sol = run(quadratic_with_lbfgs)
print('LBFGS')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

In [147]:
sol = run(quadratic_with_lbfgs)
print('LBFGS')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

In [148]:
sol = run(quadratic_with_lbfgs)
print('LBFGS')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

In [149]:
sol.position

In [150]:
st = 3
ano= 15
x1 = taus(st)
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

#optim_results = tfp.optimizer.lbfgs_minimize(
#      tflow,
#      x1,
#      num_correction_pairs=10,
#      tolerance=1e-8)


dtype= x1.dtype
start = tf.constant(sol.position, dtype=dtype)


def nelder_mead():
  return tfp.optimizer.nelder_mead_minimize(
  tflow,
  initial_vertex=start,
  parallel_iterations =10,
  shrinkage=0.9,
  func_tolerance=0)

In [151]:
sol = run(nelder_mead)
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

In [152]:
sol.position

In [153]:
dtype

In [154]:
st = 3
ano= 15
x1 = taus(st)
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

#optim_results = tfp.optimizer.lbfgs_minimize(
#      tflow,
#      x1,
#      num_correction_pairs=10,
#      tolerance=1e-8)


dtype= tf.float64
start = tf.constant(sol.position, dtype=dtype)


def nelder_mead():
  return tfp.optimizer.nelder_mead_minimize(
  tflow,
  initial_vertex=start,
  parallel_iterations =10,
  shrinkage=0.9,
  func_tolerance=0)

In [155]:
st = 3
ano= 15
x1 = taus(st)
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

#optim_results = tfp.optimizer.lbfgs_minimize(
#      tflow,
#      x1,
#      num_correction_pairs=10,
#      tolerance=1e-8)


dtype= tf.float64
start = tf.constant(np.array([-9, 0.56, 6.3]), dtype=dtype)


def nelder_mead():
  return tfp.optimizer.nelder_mead_minimize(
  tflow,
  initial_vertex=start,
  parallel_iterations =10,
  shrinkage=0.9,
  func_tolerance=0)

In [156]:
sol = run(nelder_mead)
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

In [157]:
st = 3
ano= 15
x1 = taus(st)
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

#optim_results = tfp.optimizer.lbfgs_minimize(
#      tflow,
#      x1,
#      num_correction_pairs=10,
#      tolerance=1e-8)


dtype= tf.float64
start = tf.constant(np.array([-9, 0.56, 8.3]), dtype=dtype)


def nelder_mead():
  return tfp.optimizer.nelder_mead_minimize(
  tflow,
  initial_vertex=start,
  parallel_iterations =10,
  shrinkage=0.9,
  func_tolerance=0)

In [158]:
sol = run(nelder_mead)
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

In [159]:
st = 3
ano= 15
x1 = taus(st)
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

#optim_results = tfp.optimizer.lbfgs_minimize(
#      tflow,
#      x1,
#      num_correction_pairs=10,
#      tolerance=1e-8)


dtype= tf.float64
start = tf.constant(np.array([-9, 0.56, 8.3]), dtype=dtype)


def nelder_mead():
  return tfp.optimizer.nelder_mead_minimize(
  tflow,
  initial_vertex=start,
  parallel_iterations =100,
  shrinkage=0.9,
  func_tolerance=0)

In [160]:
sol = run(nelder_mead)
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

In [161]:
st = 3
ano= 15
x1 = taus(st)
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

#optim_results = tfp.optimizer.lbfgs_minimize(
#      tflow,
#      x1,
#      num_correction_pairs=10,
#      tolerance=1e-8)


dtype= tf.float64
start = tf.constant(np.array([-9, 0.56, 8.3]), dtype=dtype)


def nelder_mead():
  return tfp.optimizer.nelder_mead_minimize(
  tflow,
  initial_vertex=start,
  parallel_iterations =100,
  shrinkage=0.9,
  func_tolerance=0,
  position_tolerance=0)

In [162]:
sol = run(nelder_mead)
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

In [163]:
st = 3
ano= 15
x1 = taus(st)
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

#optim_results = tfp.optimizer.lbfgs_minimize(
#      tflow,
#      x1,
#      num_correction_pairs=10,
#      tolerance=1e-8)


dtype= tf.float64
start = tf.constant(np.array([-9, 0.56, 8.3]), dtype=dtype)


def nelder_mead():
  return tfp.optimizer.nelder_mead_minimize(
  tflow,
  initial_vertex=start,
  parallel_iterations =100,
  shrinkage=0.9,
  func_tolerance=0,
  position_tolerance=1e-20)

In [164]:
sol = run(nelder_mead)
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

In [165]:
st = 3
ano= 15
x1 = taus(st)
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

#optim_results = tfp.optimizer.lbfgs_minimize(
#      tflow,
#      x1,
#      num_correction_pairs=10,
#      tolerance=1e-8)


dtype= tf.float64
start = tf.constant(np.array([-9, 0.56, 8.3]), dtype=dtype)


def nelder_mead():
  return tfp.optimizer.nelder_mead_minimize(
  tflow,
  initial_vertex=start,
  parallel_iterations =100,
  shrinkage=0.9,
  func_tolerance=0,
  position_tolerance=1e-10)

In [166]:
sol = run(nelder_mead)
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

In [167]:
st = 3
ano= 15
x1 = taus(st)
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

#optim_results = tfp.optimizer.lbfgs_minimize(
#      tflow,
#      x1,
#      num_correction_pairs=10,
#      tolerance=1e-8)


dtype= tf.float64
start = tf.constant(np.array([-9, 0.56, 8.3]), dtype=dtype)


def nelder_mead():
  return tfp.optimizer.nelder_mead_minimize(
  tflow,
  initial_vertex=start,
  parallel_iterations =100,
  shrinkage=0.9,
  func_tolerance=0,
  position_tolerance=1e-12)

In [168]:
sol = run(nelder_mead)
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

In [169]:
st = 3
ano= 15
x1 = taus(st)
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

#optim_results = tfp.optimizer.lbfgs_minimize(
#      tflow,
#      x1,
#      num_correction_pairs=10,
#      tolerance=1e-8)


dtype= tf.float64
start = tf.constant(np.array([-9, 0.56, 8.3]), dtype=dtype)


def nelder_mead():
  return tfp.optimizer.nelder_mead_minimize(
  tflow,
  initial_vertex=start,
  parallel_iterations =100,
  shrinkage=0.9,
  func_tolerance=0,
  position_tolerance=1e-13)

In [170]:
sol = run(nelder_mead)
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

In [171]:
st = 3
ano= 15
x1 = taus(st)
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

#optim_results = tfp.optimizer.lbfgs_minimize(
#      tflow,
#      x1,
#      num_correction_pairs=10,
#      tolerance=1e-8)


dtype= tf.float64
start = tf.constant(np.array([-5, 0.56, 2.3]), dtype=dtype)


def nelder_mead():
  return tfp.optimizer.nelder_mead_minimize(
  tflow,
  initial_vertex=start,
  parallel_iterations =100,
  shrinkage=0.9,
  func_tolerance=0,
  position_tolerance=1e-13)

In [172]:
sol = run(nelder_mead)
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

In [173]:
sol = run(nelder_mead)
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

In [174]:
st = 3
ano= 15
x1 = taus(st)
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

#optim_results = tfp.optimizer.lbfgs_minimize(
#      tflow,
#      x1,
#      num_correction_pairs=10,
#      tolerance=1e-8)


dtype= tf.float64
start = tf.constant(np.array([-5, 0.56, 8.3]), dtype=dtype)


def nelder_mead():
  return tfp.optimizer.nelder_mead_minimize(
  tflow,
  initial_vertex=start,
  parallel_iterations =100,
  shrinkage=0.9,
  func_tolerance=0,
  position_tolerance=1e-13)

In [175]:
sol = run(nelder_mead)
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

In [176]:
st = 3
ano= 15
x1 = taus(st)
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

#optim_results = tfp.optimizer.lbfgs_minimize(
#      tflow,
#      x1,
#      num_correction_pairs=10,
#      tolerance=1e-8)


dtype= tf.float64
start = tf.constant(np.array([-5, 0.56, 8.3]), dtype=dtype)


def nelder_mead():
  return tfp.optimizer.nelder_mead_minimize(
  tflow,
  initial_vertex=start,
  parallel_iterations =100,
  shrinkage=0.9,
  func_tolerance=0,
  position_tolerance=1e-14)

In [177]:
sol = run(nelder_mead)
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

In [178]:
st = 3
ano= 15
x1 = taus(st)
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

#optim_results = tfp.optimizer.lbfgs_minimize(
#      tflow,
#      x1,
#      num_correction_pairs=10,
#      tolerance=1e-8)


dtype= tf.float64
start = tf.constant(np.array([-5, 0.56, 8.3]), dtype=dtype)


def nelder_mead():
  return tfp.optimizer.nelder_mead_minimize(
  tflow,
  initial_vertex=start,
  parallel_iterations =100,
  shrinkage=0.9,
  func_tolerance=0,
  position_tolerance=1e-14)

In [179]:
sol = run(nelder_mead)
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

# -9.37196472 -8.44134184 10.66419531

In [180]:
st = 3
ano= 15
x1 = taus(st)
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

#optim_results = tfp.optimizer.lbfgs_minimize(
#      tflow,
#      x1,
#      num_correction_pairs=10,
#      tolerance=1e-8)


dtype= tf.float64
start = tf.constant(np.array([-2, 0.56, 1.3]), dtype=dtype)


def nelder_mead():
  return tfp.optimizer.nelder_mead_minimize(
  tflow,
  initial_vertex=start,
  parallel_iterations =100,
  shrinkage=0.9,
  func_tolerance=0,
  position_tolerance=1e-14)

In [181]:
sol = run(nelder_mead)
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

# -9.37196472 -8.44134184 10.66419531

In [182]:
st = 3
ano= 15
x1 = taus(st)
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

#optim_results = tfp.optimizer.lbfgs_minimize(
#      tflow,
#      x1,
#      num_correction_pairs=10,
#      tolerance=1e-8)


dtype= tf.float64
start = tf.constant(np.array([-200, 0.56, 11.3]), dtype=dtype)


def nelder_mead():
  return tfp.optimizer.nelder_mead_minimize(
  tflow,
  initial_vertex=start,
  parallel_iterations =100,
  shrinkage=0.9,
  func_tolerance=0,
  position_tolerance=1e-14)

In [183]:
sol = run(nelder_mead)
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

# -9.37196472 -8.44134184 10.66419531

In [184]:
st = 3
ano= 15
x1 = taus(st)
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

#optim_results = tfp.optimizer.lbfgs_minimize(
#      tflow,
#      x1,
#      num_correction_pairs=10,
#      tolerance=1e-8)


dtype= tf.float64
start = tf.constant(np.array([-200, 100.56, 11.3]), dtype=dtype)


def nelder_mead():
  return tfp.optimizer.nelder_mead_minimize(
  tflow,
  initial_vertex=start,
  parallel_iterations =100,
  shrinkage=0.9,
  func_tolerance=0,
  position_tolerance=1e-14)

In [185]:
sol = run(nelder_mead)
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

# -9.37196472 -8.44134184 10.66419531

In [186]:
st = 3
ano= 15
x1 = taus(st)
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

#optim_results = tfp.optimizer.lbfgs_minimize(
#      tflow,
#      x1,
#      num_correction_pairs=10,
#      tolerance=1e-8)


dtype= tf.float64
start = tf.constant(np.array([-200, 100.56, 11.3]), dtype=dtype)


def nelder_mead():
  return tfp.optimizer.nelder_mead_minimize(
  tflow,
  initial_vertex=start,
  parallel_iterations =100,
  shrinkage=0.9,
  func_tolerance=0,
  position_tolerance=1e-14)

In [187]:
sol = run(nelder_mead)
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

# -9.37196472 -8.44134184 10.66419531

In [188]:
st = 3
ano= 15
x1 = taus(st)
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

#optim_results = tfp.optimizer.lbfgs_minimize(
#      tflow,
#      x1,
#      num_correction_pairs=10,
#      tolerance=1e-8)


dtype= tf.float64
start = tf.constant(np.array([-200, 0.56, 11.3]), dtype=dtype)


def nelder_mead():
  return tfp.optimizer.nelder_mead_minimize(
  tflow,
  initial_vertex=start,
  parallel_iterations =100,
  shrinkage=0.9,
  func_tolerance=0,
  position_tolerance=1e-14)

In [189]:
sol = run(nelder_mead)
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

# -9.37196472 -8.44134184 10.66419531

In [190]:
st = 3
ano= 15
x1 = taus(st)
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

#optim_results = tfp.optimizer.lbfgs_minimize(
#      tflow,
#      x1,
#      num_correction_pairs=10,
#      tolerance=1e-8)


dtype= tf.float64
start = tf.constant(np.array([200, 0.56, 11.3]), dtype=dtype)


def nelder_mead():
  return tfp.optimizer.nelder_mead_minimize(
  tflow,
  initial_vertex=start,
  parallel_iterations =100,
  shrinkage=0.9,
  func_tolerance=0,
  position_tolerance=1e-14)

In [191]:
sol = run(nelder_mead)
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

# -9.37196472 -8.44134184 10.66419531

In [192]:
sol = run(nelder_mead)
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

# -9.37196472 -8.44134184 10.66419531

In [193]:
st = 3
ano= 15
x1 = taus(st)
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

#optim_results = tfp.optimizer.lbfgs_minimize(
#      tflow,
#      x1,
#      num_correction_pairs=10,
#      tolerance=1e-8)


dtype= tf.float64
start = tf.constant(np.array([200, 0.56, 11.3]), dtype=dtype)


def nelder_mead():
  return tfp.optimizer.nelder_mead_minimize(
  tflow,
  initial_vertex=start,
  parallel_iterations =100,
  shrinkage=0.9,
  func_tolerance=0,
  position_tolerance=1e-14)

In [194]:
sol = run(nelder_mead)
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

# -9.37196472 -8.44134184 10.66419531

In [195]:
st = 3
ano= 15
x1 = taus(st)
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

#optim_results = tfp.optimizer.lbfgs_minimize(
#      tflow,
#      x1,
#      num_correction_pairs=10,
#      tolerance=1e-8)


dtype= tf.float64
start = tf.constant(np.array([-200, 0.56, 11.3]), dtype=dtype)


def nelder_mead():
  return tfp.optimizer.nelder_mead_minimize(
  tflow,
  initial_vertex=start,
  parallel_iterations =100,
  shrinkage=0.9,
  func_tolerance=0,
  position_tolerance=1e-14)

In [196]:
sol = run(nelder_mead)
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

# -9.37196472 -8.44134184 10.66419531

In [197]:
st = 3
ano= 15
x1 = taus(st)
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

#optim_results = tfp.optimizer.lbfgs_minimize(
#      tflow,
#      x1,
#      num_correction_pairs=10,
#      tolerance=1e-8)


dtype= tf.float64
start = tf.constant(np.array([-200, 0.56, 200.3]), dtype=dtype)


def nelder_mead():
  return tfp.optimizer.nelder_mead_minimize(
  tflow,
  initial_vertex=start,
  parallel_iterations =100,
  shrinkage=0.9,
  func_tolerance=0,
  position_tolerance=1e-14)

In [198]:
sol = run(nelder_mead)
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

# -9.37196472 -8.44134184 10.66419531

In [199]:
sol = run(nelder_mead)
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

# -9.37196472 -8.44134184 10.66419531

In [200]:
st = 3
ano= 15
x1 = taus(st)
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

#optim_results = tfp.optimizer.lbfgs_minimize(
#      tflow,
#      x1,
#      num_correction_pairs=10,
#      tolerance=1e-8)


dtype= tf.float64
start = tf.constant(np.array([-200, 0.56, 200.3]), dtype=dtype)


def nelder_mead():
  return tfp.optimizer.nelder_mead_minimize(
  tflow,
  initial_vertex=start,
  parallel_iterations =100,
  shrinkage=0.9,
  func_tolerance=0,
  position_tolerance=1e-15)

In [201]:
st = 3
ano= 15
x1 = taus(st)
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

#optim_results = tfp.optimizer.lbfgs_minimize(
#      tflow,
#      x1,
#      num_correction_pairs=10,
#      tolerance=1e-8)


dtype= tf.float64
start = tf.constant(np.array([-200, 0.56, 200.3]), dtype=dtype)


def nelder_mead():
  return tfp.optimizer.nelder_mead_minimize(
  tflow,
  initial_vertex=start,
  parallel_iterations =100,
  shrinkage=0.9,
  func_tolerance=0,
  position_tolerance=1e-15)

In [202]:
sol = run(nelder_mead)
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

# -9.37196472 -8.44134184 10.66419531

In [203]:
st = 3
ano= 15
x1 = taus(st)
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

#optim_results = tfp.optimizer.lbfgs_minimize(
#      tflow,
#      x1,
#      num_correction_pairs=10,
#      tolerance=1e-8)


dtype= tf.float64
start = tf.constant(np.array([-200, 0.56, 200.3]), dtype=dtype)


def nelder_mead():
  return tfp.optimizer.nelder_mead_minimize(
  tflow,
  initial_vertex=start,
  parallel_iterations =100,
  shrinkage=0.9,
  func_tolerance=0,
  position_tolerance=1e-141)

In [204]:
sol = run(nelder_mead)
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

# -9.37196472 -8.44134184 10.66419531

In [205]:
st = 3
ano= 15
x1 = taus(st)
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

#optim_results = tfp.optimizer.lbfgs_minimize(
#      tflow,
#      x1,
#      num_correction_pairs=10,
#      tolerance=1e-8)


dtype= tf.float64
start = tf.constant(np.array([-200, 0.56, 200.3]), dtype=dtype)


def nelder_mead():
  return tfp.optimizer.nelder_mead_minimize(
  tflow,
  initial_vertex=start,
  parallel_iterations =100,
  shrinkage=0.9,
  func_tolerance=0,
  position_tolerance=1e-14.1)

In [206]:
st = 3
ano= 15
x1 = taus(st)
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

#optim_results = tfp.optimizer.lbfgs_minimize(
#      tflow,
#      x1,
#      num_correction_pairs=10,
#      tolerance=1e-8)


dtype= tf.float64
start = tf.constant(np.array([-200, 0.56, 200.3]), dtype=dtype)


def nelder_mead():
  return tfp.optimizer.nelder_mead_minimize(
  tflow,
  initial_vertex=start,
  parallel_iterations =100,
  shrinkage=0.9,
  func_tolerance=0,
  position_tolerance=1.1e-14)

In [207]:
sol = run(nelder_mead)
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

# -9.37196472 -8.44134184 10.66419531

In [208]:
sol = run(nelder_mead)
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

# -9.37196472 -8.44134184 10.66419531

In [209]:
0.9e-3

In [210]:
#0.0009

1e-3

In [211]:
st = 3
ano= 15
x1 = taus(st)
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

#optim_results = tfp.optimizer.lbfgs_minimize(
#      tflow,
#      x1,
#      num_correction_pairs=10,
#      tolerance=1e-8)


dtype= tf.float64
start = tf.constant(np.array([-200, 0.56, 200.3]), dtype=dtype)


def nelder_mead():
  return tfp.optimizer.nelder_mead_minimize(
  tflow,
  initial_vertex=start,
  parallel_iterations =100,
  shrinkage=0.9,
  func_tolerance=0,
  position_tolerance=0.9e-14)

In [212]:
sol = run(nelder_mead)
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

# -9.37196472 -8.44134184 10.66419531

In [213]:
#0.0009

In [214]:
st = 3
ano= 15
x1 = taus(st)
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

#optim_results = tfp.optimizer.lbfgs_minimize(
#      tflow,
#      x1,
#      num_correction_pairs=10,
#      tolerance=1e-8)


dtype= tf.float64
start = tf.constant(np.array([-200, 0.56, 200.3]), dtype=dtype)


def nelder_mead():
  return tfp.optimizer.nelder_mead_minimize(
  tflow,
  initial_vertex=start,
  parallel_iterations =100,
  shrinkage=0.9,
  func_tolerance=0,
  position_tolerance=0.8e-14)

In [215]:
sol = run(nelder_mead)
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

# -9.37196472 -8.44134184 10.66419531

In [216]:
st = 3
ano= 15
x1 = taus(st)
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

#optim_results = tfp.optimizer.lbfgs_minimize(
#      tflow,
#      x1,
#      num_correction_pairs=10,
#      tolerance=1e-8)


dtype= tf.float64
start = tf.constant(np.array([-200, 0.56, 200.3]), dtype=dtype)


def nelder_mead():
  return tfp.optimizer.nelder_mead_minimize(
  tflow,
  initial_vertex=start,
  parallel_iterations =100,
  shrinkage=0.9,
  func_tolerance=0,
  position_tolerance=0.7e-14)

In [217]:
sol = run(nelder_mead)
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

# -9.37196472 -8.44134184 10.66419531

In [218]:
st = 3
ano= 15
x1 = taus(st)
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

#optim_results = tfp.optimizer.lbfgs_minimize(
#      tflow,
#      x1,
#      num_correction_pairs=10,
#      tolerance=1e-8)


dtype= tf.float64
start = tf.constant(np.array([-20, 0.56, 20.3]), dtype=dtype)


def nelder_mead():
  return tfp.optimizer.nelder_mead_minimize(
  tflow,
  initial_vertex=start,
  parallel_iterations =100,
  shrinkage=0.9,
  func_tolerance=0,
  position_tolerance=0.7e-14)

In [219]:
sol = run(nelder_mead)
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

#11.37196472 10.44134184 -8.66419531

In [220]:
st = 3
ano= 15
x1 = taus(st)
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

#optim_results = tfp.optimizer.lbfgs_minimize(
#      tflow,
#      x1,
#      num_correction_pairs=10,
#      tolerance=1e-8)


dtype= tf.float64
start = tf.constant(np.array([-20, 0.56, 20.3]), dtype=dtype)


def nelder_mead():
  return tfp.optimizer.nelder_mead_minimize(
  tflow,
  initial_vertex=start,
  parallel_iterations =100,
  shrinkage=0.9,
  func_tolerance=0,
  position_tolerance=0.9e-14)

In [221]:
sol = run(nelder_mead)
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

#11.37196472 10.44134184 -8.66419531

In [222]:
st = 3
ano= 15
x1 = taus(st)
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

#optim_results = tfp.optimizer.lbfgs_minimize(
#      tflow,
#      x1,
#      num_correction_pairs=10,
#      tolerance=1e-8)


dtype= tf.float64
start = tf.constant(np.array([-20, 0.56, 20.3]), dtype=dtype)


def nelder_mead():
  return tfp.optimizer.nelder_mead_minimize(
  tflow,
  initial_vertex=start,
  parallel_iterations =100,
  shrinkage=0.3,
  func_tolerance=0,
  position_tolerance=0.9e-14)

In [223]:
sol = run(nelder_mead)
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

#-9.37196472 -8.44134184 10.66419531 # 7.878255910098147e-62

In [224]:
sol = run(quadratic_with_lbfgs)
print('LBFGS')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

In [225]:
@tf.function
def quadratic_with_lbfgs():
  return tfp.optimizer.lbfgs_minimize(
  tflow2,
  initial_position=tf.constant(np.array([-10.0, 5.0, -2.0]), dtype=dtype),
  parallel_iterations =100,
  tolerance=0,
    max_iterations=10000,
  max_line_search_iterations=5000)

In [226]:
sol = run(quadratic_with_lbfgs)
print('LBFGS')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

In [227]:
@tf.function
def quadratic_with_lbfgs():
  return tfp.optimizer.lbfgs_minimize(
  tflow2,
  initial_position=tf.constant(np.array([-10.0, 5.0, 10.0]), dtype=dtype),
  parallel_iterations =100,
  tolerance=0,
    max_iterations=10000,
  max_line_search_iterations=5000)

In [228]:
sol = run(quadratic_with_lbfgs)
print('LBFGS')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

In [229]:
@tf.function
def quadratic_with_lbfgs():
  return tfp.optimizer.lbfgs_minimize(
  tflow2,
  initial_position=tf.constant(np.array([-10.0, 5.0, 10.0]), dtype=dtype),
  parallel_iterations =10,
  tolerance=0,
    max_iterations=10000,
  max_line_search_iterations=5000)

In [230]:
sol = run(quadratic_with_lbfgs)
print('LBFGS')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

In [231]:
@tf.function
def quadratic_with_lbfgs():
  return tfp.optimizer.lbfgs_minimize(
  tflow2,
  initial_position=tf.constant(np.array([-10.0, 0.43, 10.0]), dtype=dtype),
  parallel_iterations =10,
  tolerance=0,
    max_iterations=10000,
  max_line_search_iterations=5000)

In [232]:
sol = run(quadratic_with_lbfgs)
print('LBFGS')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

In [233]:
tf.graph_util()

In [234]:
tf.graph_util

In [235]:
sol = run(quadratic_with_lbfgs)
print('LBFGS')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

In [236]:
sol = run(quadratic_with_lbfgs)
print('LBFGS')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

In [237]:
@tf.function
def quadratic_with_lbfgs():
  return tfp.optimizer.lbfgs_minimize(
  tflow2,
  initial_position=tf.constant(np.array([10.0, 0.43, 10.0]), dtype=dtype),
  parallel_iterations =10,
  tolerance=0,
    max_iterations=10000,
  max_line_search_iterations=5000)

In [238]:
sol = run(quadratic_with_lbfgs)
print('LBFGS')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

In [239]:
sol = run(quadratic_with_lbfgs)
print('LBFGS')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

In [240]:
@tf.function
def quadratic_with_lbfgs():
  return tfp.optimizer.lbfgs_minimize(
  tflow2,
  initial_position=tf.constant(np.array([10.0, 0.43, 10.0]), dtype=dtype),
  parallel_iterations =10,
 num_correction_pairs=100,
  tolerance=0,
    max_iterations=10000,
  max_line_search_iterations=5000)

In [241]:
sol = run(quadratic_with_lbfgs)
print('LBFGS')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

In [242]:
@tf.function
def quadratic_with_lbfgs():
  return tfp.optimizer.lbfgs_minimize(
  tflow2,
  initial_position=tf.constant(np.array([10.0, 0.43, 10.0]), dtype=dtype),
  parallel_iterations =10,
 num_correction_pairs=5,
  tolerance=0,
    max_iterations=10000,
  max_line_search_iterations=5000)

In [243]:
sol = run(quadratic_with_lbfgs)
print('LBFGS')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

In [244]:
@tf.function
def quadratic_with_lbfgs():
  return tfp.optimizer.lbfgs_minimize(
  tflow2,
  initial_position=tf.constant(np.array([10.0, 0.43, 10.0]), dtype=dtype),
  parallel_iterations =10,
 num_correction_pairs=5,
  tolerance=0,
    max_iterations=10000,
  max_line_search_iterations=50000)

In [245]:
sol = run(quadratic_with_lbfgs)
print('LBFGS')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

In [246]:
@tf.function
def quadratic_with_lbfgs():
  return tfp.optimizer.lbfgs_minimize(
  tflow2,
  initial_position=tf.constant(np.array([10.0, 10.0, 10.0]), dtype=dtype),
  parallel_iterations =10,
 num_correction_pairs=5,
  tolerance=0,
    max_iterations=10000,
  max_line_search_iterations=50000)

In [247]:
sol = run(quadratic_with_lbfgs)
print('LBFGS')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

In [248]:
@tf.function
def quadratic_with_lbfgs():
  return tfp.optimizer.lbfgs_minimize(
  tflow2,
  initial_position=tf.constant(np.array([-10.0, 10.0, 10.0]), dtype=dtype),
  parallel_iterations =10,
 num_correction_pairs=5,
  tolerance=0,
    max_iterations=10000,
  max_line_search_iterations=50000)

In [249]:
sol = run(quadratic_with_lbfgs)
print('LBFGS')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

In [250]:
st = 3
x1 = taus(st)
ano= 15
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

sol= minimize(tflow, x1,  method='L-BFGS-B', bounds=Bd,
              callback=callback,
              tol=1e-250, options={'maxiter':1e3}) 

In [251]:
sol.x    
 
# ----> 95 

In [252]:
sol.fun
 
# ----> 95 

In [253]:
st = 3
x1 = taus(st)
ano= 15
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

sol= minimize(tflow, x1,  method='L-BFGS-B', bounds=Bd,
              callback=callback,
              tol=0, options={'maxiter':1e3}) 

In [254]:
sol.fun
 
# ----> 95 

In [255]:
st = 3
x1 = taus(st)
ano= 15
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

sol= minimize(tflow, np.array([-10.0, 10.0, 10.0]),  method='L-BFGS-B', bounds=Bd,
              callback=callback,
              tol=0, options={'maxiter':1e3}) 

In [256]:
sol.fun
 
# ----> 95 

In [257]:
sol.x
 
# ----> 95 

In [258]:
@tf.function
def quadratic_with_lbfgs():
  return tfp.optimizer.lbfgs_minimize(
  tflow2,
  initial_position=tf.constant(np.array([-10.0, 10.0, 10.0]), dtype=dtype),
  parallel_iterations =10,
 num_correction_pairs=5,
  tolerance=0,
    max_iterations=10000,
  max_line_search_iterations=50000)

In [259]:
sol = run(quadratic_with_lbfgs)
print('LBFGS')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

In [260]:
@tf.function
def quadratic_with_lbfgs():
  return tfp.optimizer.lbfgs_minimize(
  tflow2,
  initial_position=tf.constant(np.array([10.0, 10.0, 10.0]), dtype=dtype),
  parallel_iterations =10,
 num_correction_pairs=5,
  tolerance=0,
    max_iterations=10000,
  max_line_search_iterations=50000)

In [261]:
sol = run(quadratic_with_lbfgs)
print('LBFGS')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

In [262]:
@tf.function
def quadratic_with_lbfgs():
  return tfp.optimizer.lbfgs_minimize(
  tflow2,
  initial_position=tf.constant(np.array([0.0, 0.0, 0.0]), dtype=dtype),
  parallel_iterations =10,
 num_correction_pairs=5,
  tolerance=0,
    max_iterations=10000,
  max_line_search_iterations=50000)

In [263]:
sol = run(quadratic_with_lbfgs)
print('LBFGS')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

In [264]:
3.695661944942129e-62

In [265]:
sol = run(quadratic_with_lbfgs)
print('LBFGS')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)


#-9.37196472 -8.44134184 -8.66419531
#3.695661944942129e-62

In [266]:
@tf.function
def quadratic_with_lbfgs():
  return tfp.optimizer.lbfgs_minimize(
  tflow2,
  initial_position=tf.constant(np.array([1.0, 0.0, 0.0]), dtype=dtype),
  parallel_iterations =10,
 num_correction_pairs=5,
  tolerance=0,
    max_iterations=10000,
  max_line_search_iterations=50000)

In [267]:
sol = run(quadratic_with_lbfgs)
print('LBFGS')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)


#-9.37196472 -8.44134184 -8.66419531
#3.695661944942129e-62

In [268]:
@tf.function
def quadratic_with_lbfgs():
  return tfp.optimizer.lbfgs_minimize(
  tflow2,
  initial_position=tf.constant(np.array([1.0, 0.0, 0.0]), dtype=dtype),
  parallel_iterations =10,
 num_correction_pairs=5,
  tolerance=0,
    max_iterations=10000,
  max_line_search_iterations=50000)

In [269]:
sol = run(quadratic_with_lbfgs)
print('LBFGS')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)


#-9.37196472 -8.44134184 -8.66419531
#3.695661944942129e-62

In [270]:
@tf.function
def quadratic_with_lbfgs():
  return tfp.optimizer.lbfgs_minimize(
  tflow2,
  initial_position=tf.constant(np.array([-1.0, 5.0, 5.0]), dtype=dtype),
  parallel_iterations =10,
 num_correction_pairs=5,
  tolerance=0,
    max_iterations=10000,
  max_line_search_iterations=50000)

In [271]:
sol = run(quadratic_with_lbfgs)
print('LBFGS')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)


#-9.37196472 -8.44134184 -8.66419531
#3.695661944942129e-62

In [272]:
sol = run(quadratic_with_lbfgs)
print('LBFGS')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)


#-9.37196472 -8.44134184 -8.66419531  #3.695661944942129e-62 
-9.37196472 10.44134184 10.66419531   #3.695661944942129e-62

In [273]:
sol = run(quadratic_with_lbfgs)
print('LBFGS')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)


#-9.37196472 -8.44134184 -8.66419531  #3.695661944942129e-62 
#-9.37196472 10.44134184 10.66419531   #3.695661944942129e-62

In [274]:
@tf.function
def quadratic_with_lbfgs():
  return tfp.optimizer.lbfgs_minimize(
  tflow2,
  initial_position=tf.constant(np.array([-1.0, 0.0, 5.0]), dtype=dtype),
  parallel_iterations =10,
 num_correction_pairs=5,
  tolerance=0,
    max_iterations=10000,
  max_line_search_iterations=50000)

In [275]:
sol = run(quadratic_with_lbfgs)
print('LBFGS')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)


#-9.37196472 -8.44134184 -8.66419531  #3.695661944942129e-62 
#-9.37196472 10.44134184 10.66419531   #3.695661944942129e-62

In [276]:
tf.random.uniform(maxval=10, minval=-10, shape=[1, st], dtype='float64')

In [277]:
np.random.uniform(-10, 10, s=2)

In [278]:
np.random.uniform(-10, 10, size=2)

In [279]:
np.random.uniform(-10, 10, size=3)

In [280]:


init = tf.constant(np.random.uniform(-10, 10, size=3), dtype=dtype)


sol = run(quadratic_with_lbfgs)
print('LBFGS')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)


#-9.37196472 -8.44134184 -8.66419531  #3.695661944942129e-62 
#-9.37196472 10.44134184 10.66419531   #3.695661944942129e-62

In [281]:

tol_m = 7.163690026134705e-60
init = tf.constant(np.random.uniform(-10, 10, size=3), dtype=dtype)

while val>tol_m:
    sol = run(quadratic_with_lbfgs)
    print('LBFGS')
    print('Converged:', sol.converged)
    print('Location of the minimum:', sol.position)
    print('Number of iterations:', sol.num_iterations)
    print('Objetive value:', sol.objective_value)
    val =  sol.objective_value


#-9.37196472 -8.44134184 -8.66419531  #3.695661944942129e-62 
#-9.37196472 10.44134184 10.66419531   #3.695661944942129e-62

In [282]:

tol_m = 7.163690026134705e-60
init = tf.constant(np.random.uniform(-10, 10, size=3), dtype=dtype)
val=1e-10

while val>tol_m:
    sol = run(quadratic_with_lbfgs)
    print('LBFGS')
    print('Converged:', sol.converged)
    print('Location of the minimum:', sol.position)
    print('Number of iterations:', sol.num_iterations)
    print('Objetive value:', sol.objective_value)
    val =  sol.objective_value


#-9.37196472 -8.44134184 -8.66419531  #3.695661944942129e-62 
#-9.37196472 10.44134184 10.66419531   #3.695661944942129e-62

In [283]:

tol_m = 7.163690026134705e-60
init = tf.constant(np.random.uniform(-10, 10, size=3), dtype=dtype)
val=1e-10

while val<tol_m:
    sol = run(quadratic_with_lbfgs)
    print('LBFGS')
    print('Converged:', sol.converged)
    print('Location of the minimum:', sol.position)
    print('Number of iterations:', sol.num_iterations)
    print('Objetive value:', sol.objective_value)
    val =  sol.objective_value


#-9.37196472 -8.44134184 -8.66419531  #3.695661944942129e-62 
#-9.37196472 10.44134184 10.66419531   #3.695661944942129e-62

In [284]:

tol_m = 7.163690026134705e-60
init = tf.constant(np.random.uniform(-10, 10, size=3), dtype=dtype)
val=1

while val>tol_m:
    sol = run(quadratic_with_lbfgs)
    print('LBFGS')
    print('Converged:', sol.converged)
    print('Location of the minimum:', sol.position)
    print('Number of iterations:', sol.num_iterations)
    print('Objetive value:', sol.objective_value)
    val =  sol.objective_value


#-9.37196472 -8.44134184 -8.66419531  #3.695661944942129e-62 
#-9.37196472 10.44134184 10.66419531   #3.695661944942129e-62

In [285]:
val

In [286]:

tol_m = 7.163690026134705e-62
init = tf.constant(np.random.uniform(-10, 10, size=3), dtype=dtype)
val=1

while val>tol_m:
    sol = run(quadratic_with_lbfgs)
    print('LBFGS')
    print('Converged:', sol.converged)
    print('Location of the minimum:', sol.position)
    print('Number of iterations:', sol.num_iterations)
    print('Objetive value:', sol.objective_value)
    val =  sol.objective_value


#-9.37196472 -8.44134184 -8.66419531  #3.695661944942129e-62 
#-9.37196472 10.44134184 10.66419531   #3.695661944942129e-62

In [287]:

tol_m = 7.163690026134705e-62
val=1

while val>tol_m:
    init = tf.constant(np.random.uniform(-10, 10, size=3), dtype=dtype)
    sol = run(quadratic_with_lbfgs)
    print('----LBFGS----')
    print('Converged:', sol.converged)
    print('Location of the minimum:', sol.position)
    print('Number of iterations:', sol.num_iterations)
    print('Objetive value:', sol.objective_value)
    val =  sol.objective_value


#-9.37196472 -8.44134184 -8.66419531  #3.695661944942129e-62 
#-9.37196472 10.44134184 10.66419531   #3.695661944942129e-62

In [288]:

tol_m = 7.163690026134705e-62
val=1

while val>tol_m:
    init = tf.constant(np.random.uniform(-10, 10, size=3), dtype=dtype)
    print(f'init= {init}')
    sol = run(quadratic_with_lbfgs)
    print('----LBFGS----')
    print('Converged:', sol.converged)
    print('Location of the minimum:', sol.position)
    print('Number of iterations:', sol.num_iterations)
    print('Objetive value:', sol.objective_value)
    val =  sol.objective_value


#-9.37196472 -8.44134184 -8.66419531  #3.695661944942129e-62 
#-9.37196472 10.44134184 10.66419531   #3.695661944942129e-62

In [289]:

tol_m = 7.163690026134705e-62
val=1

while val>tol_m:
    init = tf.constant(np.random.uniform(-1, 10, size=3), dtype=dtype)
    print(f'init= {init}')
    sol = run(quadratic_with_lbfgs)
    print('----LBFGS----')
    print('Converged:', sol.converged)
    print('Location of the minimum:', sol.position)
    print('Number of iterations:', sol.num_iterations)
    print('Objetive value:', sol.objective_value)
    val =  sol.objective_value


#-9.37196472 -8.44134184 -8.66419531  #3.695661944942129e-62 
#-9.37196472 10.44134184 10.66419531   #3.695661944942129e-62

In [290]:
@tf.function
def quadratic_with_lbfgs():
  return tfp.optimizer.lbfgs_minimize(
  tflow2,
print(f'init= {init}'),
  initial_position=init,
  parallel_iterations =10,
 num_correction_pairs=5,
  tolerance=0,
    max_iterations=10000,
  max_line_search_iterations=50000)

In [291]:

tol_m = 7.163690026134705e-62
val=1

while val>tol_m:
    init = tf.constant(np.random.uniform(-1, 10, size=3), dtype=dtype)
   
    sol = run(quadratic_with_lbfgs)
    print('----LBFGS----')
    print('Converged:', sol.converged)
    print('Location of the minimum:', sol.position)
    print('Number of iterations:', sol.num_iterations)
    print('Objetive value:', sol.objective_value)
    val =  sol.objective_value


#-9.37196472 -8.44134184 -8.66419531  #3.695661944942129e-62 
#-9.37196472 10.44134184 10.66419531   #3.695661944942129e-62

In [292]:
@tf.function
def quadratic_with_lbfgs():
     print(f'init= {init}')
  return tfp.optimizer.lbfgs_minimize(
  tflow2,
  initial_position=init,
  parallel_iterations =10,
 num_correction_pairs=5,
  tolerance=0,
    max_iterations=10000,
  max_line_search_iterations=50000)

In [293]:
@tf.function
def quadratic_with_lbfgs():
  return tfp.optimizer.lbfgs_minimize(
  tflow2,
  initial_position=init,
  parallel_iterations =10,
 num_correction_pairs=5,
  tolerance=0,
    max_iterations=10000,
  max_line_search_iterations=50000)
    print(f'init= {init}')

In [294]:
@tf.function
def quadratic_with_lbfgs():
  return tfp.optimizer.lbfgs_minimize(
  tflow2,
  initial_position=init,
  parallel_iterations =10,
  num_correction_pairs=5,
  tolerance=0,
  max_iterations=10000,
  max_line_search_iterations=50000)
  print(f'init= {init}')

In [295]:

tol_m = 7.163690026134705e-62
val=1

while val>tol_m:
    init = tf.constant(np.random.uniform(-1, 10, size=3), dtype=dtype)
   
    sol = run(quadratic_with_lbfgs)
    print('----LBFGS----')
    print('Converged:', sol.converged)
    print('Location of the minimum:', sol.position)
    print('Number of iterations:', sol.num_iterations)
    print('Objetive value:', sol.objective_value)
    val =  sol.objective_value


#-9.37196472 -8.44134184 -8.66419531  #3.695661944942129e-62 
#-9.37196472 10.44134184 10.66419531   #3.695661944942129e-62

In [296]:
@tf.function
def quadratic_with_lbfgs(init):
  return tfp.optimizer.lbfgs_minimize(
  tflow2,
  initial_position=init,
  parallel_iterations =10,
  num_correction_pairs=5,
  tolerance=0,
  max_iterations=10000,
  max_line_search_iterations=50000)

In [297]:

tol_m = 7.163690026134705e-62
val=1

while val>tol_m:
    init = tf.constant(np.random.uniform(-1, 10, size=3), dtype=dtype)
    sol = run(quadratic_with_lbfgs(init))
    print('----LBFGS----')
    print('Converged:', sol.converged)
    print('Location of the minimum:', sol.position)
    print('Number of iterations:', sol.num_iterations)
    print('Objetive value:', sol.objective_value)
    val =  sol.objective_value


#-9.37196472 -8.44134184 -8.66419531  #3.695661944942129e-62 
#-9.37196472 10.44134184 10.66419531   #3.695661944942129e-62

In [298]:

tol_m = 7.163690026134705e-62
val=1

while val>tol_m:
    init = tf.constant(np.random.uniform(-1, 10, size=3), dtype=dtype)
    sol = run(quadratic_with_lbfgs)
    print('----LBFGS----')
    print('Converged:', sol.converged)
    print('Location of the minimum:', sol.position)
    print('Number of iterations:', sol.num_iterations)
    print('Objetive value:', sol.objective_value)
    val =  sol.objective_value


#-9.37196472 -8.44134184 -8.66419531  #3.695661944942129e-62 
#-9.37196472 10.44134184 10.66419531   #3.695661944942129e-62

In [299]:

tol_m = 7.163690026134705e-62
val=1

while val>tol_m:
    init = tf.constant(np.random.uniform(-1, 10, size=3), dtype=dtype)
    sol = run(quadratic_with_lbfgs(init))
    print('----LBFGS----')
    print('Converged:', sol.converged)
    print('Location of the minimum:', sol.position)
    print('Number of iterations:', sol.num_iterations)
    print('Objetive value:', sol.objective_value)
    val =  sol.objective_value


#-9.37196472 -8.44134184 -8.66419531  #3.695661944942129e-62 
#-9.37196472 10.44134184 10.66419531   #3.695661944942129e-62

In [300]:
def make_val_and_grad_fn(value_fn):
  @functools.wraps(value_fn)
  def val_and_grad(x):
    return tfp.math.value_and_gradient(value_fn, x)
  return val_and_grad



@contextlib.contextmanager
def timed_execution():
  t0 = time.time()
  yield
  dt = time.time() - t0
  print('Evaluation took: %f seconds' % dt)


def np_value(tensor):
  """Get numpy value out of possibly nested tuple of tensors."""
  if isinstance(tensor, tuple):
    return type(tensor)(*(np_value(t) for t in tensor))
  else:
    return tensor.numpy()

def run(optimizer):
  """Run an optimizer and measure it's evaluation time."""
  optimizer(x)  # Warmup.
  with timed_execution():
    result = optimizer(x)
  return np_value(result)

In [301]:

tol_m = 7.163690026134705e-62
val=1

while val>tol_m:
    init = tf.constant(np.random.uniform(-1, 10, size=3), dtype=dtype)
    sol = run(quadratic_with_lbfgs(init))
    print('----LBFGS----')
    print('Converged:', sol.converged)
    print('Location of the minimum:', sol.position)
    print('Number of iterations:', sol.num_iterations)
    print('Objetive value:', sol.objective_value)
    val =  sol.objective_value


#-9.37196472 -8.44134184 -8.66419531  #3.695661944942129e-62 
#-9.37196472 10.44134184 10.66419531   #3.695661944942129e-62

In [302]:
def make_val_and_grad_fn(value_fn):
  @functools.wraps(value_fn)
  def val_and_grad(x):
    return tfp.math.value_and_gradient(value_fn, x)
  return val_and_grad



@contextlib.contextmanager
def timed_execution():
  t0 = time.time()
  yield
  dt = time.time() - t0
  print('Evaluation took: %f seconds' % dt)


def np_value(tensor):
  """Get numpy value out of possibly nested tuple of tensors."""
  if isinstance(tensor, tuple):
    return type(tensor)(*(np_value(t) for t in tensor))
  else:
    return tensor.numpy()

def run(optimizer):
  """Run an optimizer and measure it's evaluation time."""
  optimizer()  # Warmup.
  with timed_execution():
    result = optimizer()
  return np_value(result)

In [303]:

tol_m = 7.163690026134705e-62
val=1

while val>tol_m:
    init = tf.constant(np.random.uniform(-1, 10, size=3), dtype=dtype)
    sol = run(quadratic_with_lbfgs(init))
    print('----LBFGS----')
    print('Converged:', sol.converged)
    print('Location of the minimum:', sol.position)
    print('Number of iterations:', sol.num_iterations)
    print('Objetive value:', sol.objective_value)
    val =  sol.objective_value


#-9.37196472 -8.44134184 -8.66419531  #3.695661944942129e-62 
#-9.37196472 10.44134184 10.66419531   #3.695661944942129e-62

In [304]:

tol_m = 7.163690026134705e-62
val=1

while val>tol_m:
    init = tf.constant(np.random.uniform(-1, 10, size=3), dtype=dtype)
    sol = quadratic_with_lbfgs(init)
    print('----LBFGS----')
    print('Converged:', sol.converged)
    print('Location of the minimum:', sol.position)
    print('Number of iterations:', sol.num_iterations)
    print('Objetive value:', sol.objective_value)
    val =  sol.objective_value


#-9.37196472 -8.44134184 -8.66419531  #3.695661944942129e-62 
#-9.37196472 10.44134184 10.66419531   #3.695661944942129e-62

In [305]:

tol_m = 7.163690026134705e-62
val=1

while val>tol_m:
    init = tf.constant(np.random.uniform(-1, 10, size=3), dtype=dtype)
    sol = np_value(quadratic_with_lbfgs(init))
    print('----LBFGS----')
    print('Converged:', sol.converged)
    print('Location of the minimum:', sol.position)
    print('Number of iterations:', sol.num_iterations)
    print('Objetive value:', sol.objective_value)
    val =  sol.objective_value


#-9.37196472 -8.44134184 -8.66419531  #3.695661944942129e-62 
#-9.37196472 10.44134184 10.66419531   #3.695661944942129e-62

In [306]:

tol_m = 7.163690026134705e-65
val=1

while val>tol_m:
    init = tf.constant(np.random.uniform(-1, 10, size=3), dtype=dtype)
    sol = np_value(quadratic_with_lbfgs(init))
    print('----LBFGS----')
    print('Converged:', sol.converged)
    print('Location of the minimum:', sol.position)
    print('Number of iterations:', sol.num_iterations)
    print('Objetive value:', sol.objective_value)
    val =  sol.objective_value


#-9.37196472 -8.44134184 -8.66419531  #3.695661944942129e-62 
#-9.37196472 10.44134184 10.66419531   #3.695661944942129e-62

In [307]:

tol_m = 7.163690026134705e-65
val=1

while val>tol_m:
    init = tf.constant(np.random.uniform(-10, 10, size=3), dtype=dtype)
    sol = np_value(quadratic_with_lbfgs(init))
    print('----LBFGS----')
    print(f'init = {init}')
    print('Converged:', sol.converged)
    print('Location of the minimum:', sol.position)
    print('Number of iterations:', sol.num_iterations)
    print('Objetive value:', sol.objective_value)
    val =  sol.objective_value


#-9.37196472 -8.44134184 -8.66419531  #3.695661944942129e-62 
#-9.37196472 10.44134184 10.66419531   #3.695661944942129e-62

In [308]:

tol_m = 5e-63
val=1

while val>tol_m:
    init = tf.constant(np.random.uniform(-10, 10, size=3), dtype=dtype)
    sol = np_value(quadratic_with_lbfgs(init))
    print('----LBFGS----')
    print(f'init = {init}')
    print('Converged:', sol.converged)
    print('Location of the minimum:', sol.position)
    print('Number of iterations:', sol.num_iterations)
    print('Objetive value:', sol.objective_value)
    val =  sol.objective_value


#-9.37196472 -8.44134184 -8.66419531  #3.695661944942129e-62 
#-9.37196472 10.44134184 10.66419531   #3.695661944942129e-62

In [309]:
@tf.function
def quadratic_with_lbfgs(init):
  return tfp.optimizer.lbfgs_minimize(
  tflow2,
  initial_position=init,
  parallel_iterations =10,
  num_correction_pairs=5,
  tolerance=0,
  max_iterations=10000,
  max_line_search_iterations=50000)

print(10*'-' + 'LBFGS' + 10*'-')

In [310]:
@tf.function
def quadratic_with_lbfgs(init):
  return tfp.optimizer.lbfgs_minimize(
  tflow2,
  initial_position=init,
  parallel_iterations =10,
  num_correction_pairs=5,
  tolerance=0,
  max_iterations=10000,
  max_line_search_iterations=50000)

In [311]:

tol_m = 5e-62
val=1

while val>tol_m:
    init = tf.constant(np.random.uniform(-10, 10, size=3), dtype=dtype)
    sol = np_value(quadratic_with_lbfgs(init))
    print(15*'-' + 'LBFGS' + 15*'-')
    print(f'init = {init}')
    print('Converged:', sol.converged)
    print('Location of the minimum:', sol.position)
    print('Number of iterations:', sol.num_iterations)
    print('Objetive value:', sol.objective_value)
    val =  sol.objective_value


#-9.37196472 -8.44134184 -8.66419531  #3.695661944942129e-62 
#-9.37196472 10.44134184 10.66419531   #3.695661944942129e-62

In [312]:

tol_m = 3e-62
val=1

while val>tol_m:
    init = tf.constant(np.random.uniform(-10, 10, size=3), dtype=dtype)
    sol = np_value(quadratic_with_lbfgs(init))
    print(15*'-' + 'LBFGS' + 15*'-')
    print(f'init = {init}')
    print('Converged:', sol.converged)
    print('Location of the minimum:', sol.position)
    print('Number of iterations:', sol.num_iterations)
    print('Objetive value:', sol.objective_value)
    val =  sol.objective_value


#-9.37196472 -8.44134184 -8.66419531  #3.695661944942129e-62 
#-9.37196472 10.44134184 10.66419531   #3.695661944942129e-62

In [313]:

tol_m = 3.5e-62
val=1

while val>tol_m:
    init = tf.constant(np.random.uniform(-10, 10, size=3), dtype=dtype)
    sol = np_value(quadratic_with_lbfgs(init))
    print(15*'-' + 'LBFGS' + 15*'-')
    print(f'init = {init}')
    print('Converged:', sol.converged)
    print('Location of the minimum:', sol.position)
    print('Number of iterations:', sol.num_iterations)
    print('Objetive value:', sol.objective_value)
    val =  sol.objective_value


#-9.37196472 -8.44134184 -8.66419531  #3.695661944942129e-62 
#-9.37196472 10.44134184 10.66419531   #3.695661944942129e-62

In [314]:

tol_m = 3e-62
val=1

while val>tol_m:
    init = tf.constant(np.random.uniform(-10, 10, size=3), dtype=dtype)
    sol = np_value(quadratic_with_lbfgs(init))
    print(15*'-' + 'LBFGS' + 15*'-')
    print(f'init = {init}')
    print('Converged:', sol.converged)
    print('Location of the minimum:', sol.position)
    print('Number of iterations:', sol.num_iterations)
    print('Objetive value:', sol.objective_value)
    val =  sol.objective_value

In [315]:

tol_m = 3.5e-62
val=1

while val>tol_m:
    init = tf.constant(np.random.uniform(-10, 10, size=3), dtype=dtype)
    sol = np_value(quadratic_with_lbfgs(init))
    print(15*'-' + 'LBFGS' + 15*'-')
    print(f'init = {init}')
    print('Converged:', sol.converged)
    print('Location of the minimum:', sol.position)
    print('Number of iterations:', sol.num_iterations)
    print('Objetive value:', sol.objective_value)
    val =  sol.objective_value

In [316]:
@tf.function
def quadratic_with_lbfgs(init):
  return tfp.optimizer.lbfgs_minimize(
  tflow2,
  initial_position=init,
  parallel_iterations =100,
  num_correction_pairs=5,
  tolerance=0,
  max_iterations=10000,
  max_line_search_iterations=50000)

In [317]:

tol_m = 3.5e-62
val=1

while val>tol_m:
    init = tf.constant(np.random.uniform(-10, 10, size=3), dtype=dtype)
    sol = np_value(quadratic_with_lbfgs(init))
    print(15*'-' + 'LBFGS' + 15*'-')
    print(f'init = {init}')
    print('Converged:', sol.converged)
    print('Location of the minimum:', sol.position)
    print('Number of iterations:', sol.num_iterations)
    print('Objetive value:', sol.objective_value)
    val =  sol.objective_value

In [318]:

tol_m = 4.5e-62
val=1
c = 0
while val>tol_m:
    c += 1
    init = tf.constant(np.random.uniform(-10, 10, size=3), dtype=dtype)
    sol = np_value(quadratic_with_lbfgs(init))
    print(15*'-' + 'LBFGS' + 15*'-'+ f('Iteração: {c}'))
    print(f'init = {init}')
    print('Converged:', sol.converged)
    print('Location of the minimum:', sol.position)
    print('Number of iterations:', sol.num_iterations)
    print('Objetive value:', sol.objective_value)
    val =  sol.objective_value

In [319]:

tol_m = 4.5e-62
val=1
c = 0
while val>tol_m:
    c += 1
    init = tf.constant(np.random.uniform(-10, 10, size=3), dtype=dtype)
    sol = np_value(quadratic_with_lbfgs(init))
    print(15*'-' + 'LBFGS' + 15*'-'+ (f'Iteração: {c}'))
    print(f'init = {init}')
    print('Converged:', sol.converged)
    print('Location of the minimum:', sol.position)
    print('Number of iterations:', sol.num_iterations)
    print('Objetive value:', sol.objective_value)
    val =  sol.objective_value

In [320]:

tol_m = 4e-62
val=1
c = 0
while val>tol_m:
    c += 1
    init = tf.constant(np.random.uniform(-10, 10, size=3), dtype=dtype)
    sol = np_value(quadratic_with_lbfgs(init))
    print(15*'-' + 'LBFGS' + 15*'-'+ ' ' + (f'Iteração: {c}'))
    print(f'init = {init}')
    print('Converged:', sol.converged)
    print('Location of the minimum:', sol.position)
    print('Number of iterations:', sol.num_iterations)
    print('Objetive value:', sol.objective_value)
    val =  sol.objective_value

In [321]:

tol_m = 3.695661944942129e-62
val=1
c = 0
while val>tol_m:
    c += 1
    init = tf.constant(np.random.uniform(-10, 10, size=3), dtype=dtype)
    sol = np_value(quadratic_with_lbfgs(init))
    print(15*'-' + 'LBFGS' + 15*'-'+ ' ' + (f'Iteração: {c}'))
    print(f'init = {init}')
    print('Converged:', sol.converged)
    print('Location of the minimum:', sol.position)
    print('Number of iterations:', sol.num_iterations)
    print('Objetive value:', sol.objective_value)
    val =  sol.objective_value

In [322]:

tol_m = 3.68e-62

val=1
c = 0
while val>tol_m:
    c += 1
    init = tf.constant(np.random.uniform(-10, 10, size=3), dtype=dtype)
    sol = np_value(quadratic_with_lbfgs(init))
    print(15*'-' + 'LBFGS' + 15*'-'+ ' ' + (f'Iteração: {c}'))
    print(f'init = {init}')
    print('Converged:', sol.converged)
    print('Location of the minimum:', sol.position)
    print('Number of iterations:', sol.num_iterations)
    print('Objetive value:', sol.objective_value)
    val =  sol.objective_value

In [323]:
@make_val_and_grad_fn
def tflow2(x1):    
    global w   
    w=lin_sys(x1)
    x1=tf.reshape(x1, (1, 3))
    tf.convert_to_tensor(x1, tf.float64)
    W_im = W_i_f(x1)[0]    
    D = 100000*tf.reduce_sum(tf.pow( tf.divide(tf.math.squared_difference(W_id, W_im), W_id ) , 2)) 
    #D = tf.math.log(D)
    return D

In [324]:
@tf.function
def quadratic_with_lbfgs(init):
  return tfp.optimizer.lbfgs_minimize(
  tflow2,
  initial_position=init,
  parallel_iterations =100,
  num_correction_pairs=5,
  tolerance=0,
  max_iterations=10000,
  max_line_search_iterations=50000)

In [325]:

tol_m = 3.58e-62

val=1
c = 0
while val>tol_m:
    c += 1
    init = tf.constant(np.random.uniform(-10, 10, size=3), dtype=dtype)
    sol = np_value(quadratic_with_lbfgs(init))
    print(15*'-' + 'LBFGS' + 15*'-'+ ' ' + (f'Iteração: {c}'))
    print(f'init = {init}')
    print('Converged:', sol.converged)
    print('Location of the minimum:', sol.position)
    print('Number of iterations:', sol.num_iterations)
    print('Objetive value:', sol.objective_value)
    val =  sol.objective_value

In [326]:
st = 3
ano= 15
x1 = taus(st)
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

#optim_results = tfp.optimizer.lbfgs_minimize(
#      tflow,
#      x1,
#      num_correction_pairs=10,
#      tolerance=1e-8)


dtype= tf.float64
start = tf.constant(np.array([-20, 0.56, 20.3]), dtype=dtype)


def nelder_mead():
  return tfp.optimizer.nelder_mead_minimize(
  tflow,
  initial_vertex=start,
  parallel_iterations =100,
  shrinkage=0.3,
  func_tolerance=0,
  position_tolerance=0.9e-14)

In [327]:
sol = run(nelder_mead)
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

#-9.37196472 -8.44134184 10.66419531 # 7.878255910098147e-62

In [328]:
sol = run(nelder_mead)
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

#-9.37196472 -8.44134184 10.66419531 # 7.878255910098147e-62

In [329]:
def N_if(x1):
    S = tf.pow(s, phi*(theta*lamb+ kappa) )
    S2 =tf.pow((1 - s), ( (theta*lamb)/(beta*kappa) ) )
    H = tf.pow((1 + x1[0]), (theta*lamb + kappa))    
    C = tf.multiply(tf.multiply(tf.pow(psi, theta*lamb), eta**gamma1 ), G )
    N_i = tf.multiply(S, tf.multiply(S2, tf.multiply(H, C)) )
    return N_i

In [330]:
def w_til_f(x1):
    #w = lin_sys(x1)
    #print(f'w = {w}')
    S = tf.pow(s, phi)
    S2 = tf.pow( (1 - s), 1/(beta*kappa) )
    H = 1 + x1[0]
    w_til = tf.multiply(psi, tf.multiply(H, tf.multiply(S, tf.multiply(w , S2))))
    return w_til

   

In [331]:
sol = run(nelder_mead)
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

#-9.37196472 -8.44134184 10.66419531 # 7.878255910098147e-62

In [332]:
sol = run(nelder_mead)
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

#-9.37196472 -8.44134184 10.66419531 # 7.878255910098147e-62

In [333]:
sol = run(nelder_mead)
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

#-11.37196472   8.44134184   8.66419531 # 4.231891940474952e-62

In [334]:
st = 3
ano= 15
x1 = taus(st)
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0



dtype= tf.float64
start = tf.constant(np.array([20, 0.56, 20.3]), dtype=dtype)


def nelder_mead():
  return tfp.optimizer.nelder_mead_minimize(
  tflow,
  initial_vertex=start,
  parallel_iterations =100,
  shrinkage=0.3,
  func_tolerance=0,
  position_tolerance=0.9e-14)

In [335]:
sol = run(nelder_mead)
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

#-11.37196472   8.44134184   8.66419531 # 4.231891940474952e-62

In [336]:
st = 3
ano= 15
x1 = taus(st)
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0



dtype= tf.float64
start = tf.constant(np.array([10, -0.56, 20.3]), dtype=dtype)


def nelder_mead():
  return tfp.optimizer.nelder_mead_minimize(
  tflow,
  initial_vertex=start,
  parallel_iterations =100,
  shrinkage=0.3,
  func_tolerance=0,
  position_tolerance=0.9e-14)

In [337]:
sol = run(nelder_mead)
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

#-11.37196472   8.44134184   8.66419531 # 4.231891940474952e-62

In [338]:
sol = run(nelder_mead)
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

#-11.37196472   8.44134184   8.66419531 # 4.231891940474952e-62

In [339]:
st = 3
ano= 15
x1 = taus(st)
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0



dtype= tf.float64
start = tf.constant(np.array([10, -0.56, -20.3]), dtype=dtype)


def nelder_mead():
  return tfp.optimizer.nelder_mead_minimize(
  tflow,
  initial_vertex=start,
  parallel_iterations =100,
  shrinkage=0.3,
  func_tolerance=0,
  position_tolerance=0.9e-14)

In [340]:
sol = run(nelder_mead)
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

#-11.37196472   8.44134184   8.66419531 # 4.231891940474952e-62

In [341]:
st = 3
ano= 15
x1 = taus(st)
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0



dtype= tf.float64
start = tf.constant(np.array([0, -0.0, 0.3]), dtype=dtype)


def nelder_mead():
  return tfp.optimizer.nelder_mead_minimize(
  tflow,
  initial_vertex=start,
  parallel_iterations =100,
  shrinkage=0.3,
  func_tolerance=0,
  position_tolerance=0.9e-14)

In [342]:
sol = run(nelder_mead)
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

#-11.37196472   8.44134184   8.66419531 # 4.231891940474952e-62

In [343]:
st = 3
ano= 15
x1 = taus(st)
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0



dtype= tf.float64
start = tf.constant(np.array([0, -0.0, 0.3]), dtype=dtype)


def nelder_mead():
  return tfp.optimizer.nelder_mead_minimize(
  tflow,
  initial_vertex=start,
  parallel_iterations =100,
  shrinkage=0.3,
  func_tolerance=0,
  position_tolerance=0.9e-14)

In [344]:
sol = run(nelder_mead)
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

#-11.37196472   8.44134184   8.66419531 # 4.231891940474952e-62

In [345]:
st = 3
ano= 15
x1 = taus(st)
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0



dtype= tf.float64
start = tf.constant(np.array([0, -0.0, 0.0]), dtype=dtype)


def nelder_mead():
  return tfp.optimizer.nelder_mead_minimize(
  tflow,
  initial_vertex=start,
  parallel_iterations =100,
  shrinkage=0.3,
  func_tolerance=0,
  position_tolerance=0.9e-14)

In [346]:
sol = run(nelder_mead)
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

#-11.37196472   8.44134184   8.66419531 # 4.231891940474952e-62

In [347]:
st = 3
ano= 15
x1 = taus(st)
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0



dtype= tf.float64
start = tf.constant(np.array([0.0, 1.0, 0.0]), dtype=dtype)


def nelder_mead():
  return tfp.optimizer.nelder_mead_minimize(
  tflow,
  initial_vertex=start,
  parallel_iterations =100,
  shrinkage=0.3,
  func_tolerance=0,
  position_tolerance=0.9e-14)

In [348]:
sol = run(nelder_mead)
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

#-11.37196472   8.44134184   8.66419531 # 4.231891940474952e-62

In [349]:
st = 3
ano= 15
x1 = taus(st)
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0



dtype= tf.float64
start = tf.constant(np.array([0.0, 0.0, 0.0]), dtype=dtype)


def nelder_mead():
  return tfp.optimizer.nelder_mead_minimize(
  tflow,
  initial_vertex=start,
  parallel_iterations =100,
  shrinkage=0.3,
  func_tolerance=0,
  position_tolerance=0.9e-14)

In [350]:
sol = run(nelder_mead)
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

#-11.37196472   8.44134184   8.66419531 # 4.231891940474952e-62

In [351]:
st = 3
ano= 15
x1 = taus(st)
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0



dtype= tf.float64
start = tf.constant(np.array([1.0, 3.10, -1.20]), dtype=dtype)


def nelder_mead():
  return tfp.optimizer.nelder_mead_minimize(
  tflow,
  initial_vertex=start,
  parallel_iterations =100,
  shrinkage=0.3,
  func_tolerance=0,
  position_tolerance=0.9e-14)

In [352]:
sol = run(nelder_mead)
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

#-11.37196472   8.44134184   8.66419531 # 4.231891940474952e-62

In [353]:
sol = run(nelder_mead)
print('Nelder-Mead')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

#-11.37196472   8.44134184   8.66419531 # 4.231891940474952e-62

In [354]:
sol = run(nelder_mead)

print(15*'-' + 'Nelder-Mead' + 15*'-'+ ' ' + (f'Iteração: {c}'))
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

#-11.37196472   8.44134184   8.66419531 # 4.231891940474952e-62

In [355]:
sol = run(nelder_mead)

print(15*'-' + 'Nelder-Mead' + 15*'-')
print('Converged:', sol.converged)
print('Location of the minimum:', sol.position)
print('Number of iterations:', sol.num_iterations)
print('Objetive value:',sol.objective_value)

#-11.37196472   8.44134184   8.66419531 # 4.231891940474952e-62